## Finding similar articles

In [ ]:
#Adapted from this tutorial
#https://towardsdatascience.com/a-laymans-guide-to-fuzzy-document-deduplication-a3b3cf9a05a7

In [1]:
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, Phraser
from gensim import corpora
from gensim.similarities import Similarity
import pandas as pd

In [3]:
#Getting the headlines

data = pd.read_csv('december_full_data.csv')
data.shape

(33380, 13)

In [3]:
#For the purposes of looking for similar articles, we cannot look at all 30k+ articles at the same time because 
#of memory contraints. I will order the articles by date and then separate it into chunks of 5,000 to look for similar
#articles. Similar articles should appear together in the 5,000 article blocks

data = data.sort_values(by='date', ascending=True)

In [4]:
headlines = []

for x in data['title']:
    headlines.append(x)

In [5]:
headlines

['VIDEO: Middle Ground: A conversation between an Israeli and a Palestinian',
 'FOX4 News Team',
 'INTERACTIVE – Webster recruiting',
 'Daughter of Webster University’s first black student shares mother’s story',
 'AUDIO SLIDESHOW: Webster Groves churches host vigil, support same-sex marriage',
 'Subscriptions to the Caldwell County News',
 'MU, Stephens College, Columbia College graduations are this weekend',
 'Real With Tennille Interview With Byron Smith owner Instructor of Metro Barber College',
 'Oprah Brown Interviews D’Juan Love founder of Kansas City Black Businesses',
 'Rodney Smith, Ph D, Associate Director, UMKC Academic Development',
 'Oprah Brown Interviews Local Rappers T Money & Bird Brown',
 'Oprah Brown interviews Members of Go Hard, Darockboi and DC',
 'Honor God by Honoring the Man of God',
 'Six things to know about finishing well',
 'Risk of American ‘megadroughts’ for decades, NASA warns',
 'Mike Pound: Col. Travis Hoover among those honored for role in World War 

In [4]:
#Need to make sure all of the entries in headlines are strings

for x in headlines:
    if not isinstance(x, str):
        print(x)
        
headlines_cleaned = []
        
headlines_cleaned[:] = [x for x in headlines if isinstance(x,str)]

nan
nan
nan
nan
nan


In [5]:
documents = headlines_cleaned[30001:]
len(documents)

3374

In [6]:
#Converting documents to collection of words
#Deacc=True strips the punctuation accents and numbers

texts = [[text for text in simple_preprocess(doc, deacc=True)] for doc in documents]

In [7]:
#Build a bigram model -- but using the Gensim library, which only stores common bigrams (saves on time and space)

bigram = Phrases(texts, min_count=1)
bigram_phraser = Phraser(bigram)

In [8]:
#Reconvert documents to collection of words/bigrams
texts_bigrams = [[text for text in bigram_phraser[ simple_preprocess(doc, deacc=True)]] for doc in documents]

In [9]:
#Using a Gensim dictionary
dictionary = corpora.Dictionary(texts_bigrams)

In [10]:
#Creating a Gensim corpus
#Basically uses numbers which speeds up the process substantially
corpus = [dictionary.doc2bow(docString) for docString in texts_bigrams]

In [11]:
#Build similarity index
#Calculates the pairwise cosine similarity of each document
index = Similarity(corpus=corpus,
                   num_features=len(dictionary),
                   output_prefix='on_disk_output')

In [12]:
doc_id = 0
similar_docs = {}
for similarities in index:
    similar_docs[doc_id] = list(enumerate(similarities))
    doc_id += 1

In [13]:
#Setting a similarity threshold
sim_threshold = 0.9

In [14]:
#Finding similar documents

exact_matches = []

for doc_id, sim_doc_tuples in similar_docs.items():
    for sim_doc_tuple in sim_doc_tuples:
        sim_doc_id = sim_doc_tuple[0]
        sim_score = sim_doc_tuple[1]
        if sim_score >= sim_threshold and doc_id != sim_doc_id and sim_score==1:
            exact_matches.append(documents[doc_id])
            exact_matches.append(documents[sim_doc_id])
            
            
            

In [15]:
exact_matches = list(set(exact_matches))
exact_matches

["DONNA ERICKSON'S BEST BITES: Serve up Caesar salad for whole family",
 '‘Atmospheric river’ dumps heavy rain, snow across California',
 'Border crisis: Shootouts, pursuits, break-ins surge in rural Texas counties',
 'Chiefs: Mahomes reaches 5000 yards passing once again',
 'Elon Musk buying Twitter has been all over the news lately. Are you on Twitter?',
 'Stable Chiefs face reeling Broncos in midst of playoff push',
 "Taxes fall, wages rise and jaywalking OK'd by new state laws",
 'Time zone by time zone, another new year sweeps into view',
 'Newsletters',
 'East St. Louis man shot dead in Carr Square',
 'US to require travelers from China to show negative Covid-19 test result before flight',
 'Despite changes, Broncos stuck with same old result in KC',
 'No. 1 Georgia rallies to beat No. 4 Ohio State 42-41 in semi',
 'What they said: A look back at the interviews of 2022',
 'Falcons WR Cameron Batson facing five charges',
 'Kirstie Alley, Emmy-winning ‘Cheers’ star, dies at 71',
 "

In [16]:
close_matches = []

for doc_id, sim_doc_tuples in similar_docs.items():
    for sim_doc_tuple in sim_doc_tuples:
        sim_doc_id = sim_doc_tuple[0]
        sim_score = sim_doc_tuple[1]
        if 0.9 <= sim_score < 0.99 and doc_id != sim_doc_id:
            close_matches.append(documents[doc_id])
            close_matches.append(documents[sim_doc_id])

In [17]:
close_matches = list(set(close_matches))
close_matches

['Capitol Perspectives: The 50th Anniversary of Missouri’s Sunshine Law',
 'A record-setting number of women will serve in state legislatures in 2023 • Missouri Independent',
 'New York OKs human composting law; 6th state in US to do so',
 'Legacy of Mathews celebrated by the many lives he touched',
 'Cole County judge blocks effort to deny Medicaid funds to Planned Parenthood • Missouri Independent',
 'Legacy of Mathews celebrated by many lives he touched',
 'White House resumes handing out free COVID-19 rapid test kits',
 'White House resumes handing out free COVID-19 rapid test kits • Missouri Independent',
 'How to stay proactive about your health with multi-cancer early detection screening',
 "The 50th Anniversary of Missouri's Sunshine Law",
 'A record-setting number of women will serve in state legislatures in 2023',
 'New York OKs human composting, becomes 6th state in US to do so',
 'Stay proactive about your health with multi-cancer early detection screening',
 'Cole County j

## Exact Duplicates

In [3]:


exact_duplicates = ['FBI director raises national security concerns about TikTok',
 'Tools for Teachers Presented by Weber Chevrolet',
 'Taking on the role of Thanksgiving host',
 'Police report 11-30-22',
 'Christine McVie, Fleetwood Mac singer-songwriter, dies at 79',
 'Speak Out 12-2-22',
 'MDC reports 197,724 deer harvested during November portion of firearms deer season',
 'Fire report 11-30-22',
 'R-I School Menu 12-1-22',
 "Olivia Colman 'incredibly proud' of Kit Connor",
 'Letter to the Editor – Chillicothe News Constitution-Tribune',
 '2022 4-H Week',
 'Widespread outage shuts down 988 mental health hotline',
 "Guilty plea in boy's death that sparked federal task force",
 'Snowden receives Russian passport, takes citizenship oath',
 'SPRING SPORTS 2021',
 'US player Pulisic cleared to play against Dutch in World Cup',
 'MCR-1 honors student-athletes at annual banquet',
 'McGaugh recognizes Pearson family during car show',
 'Bill proposed to implement minimum age to possess firearm in Missouri',
 "A judge orders Arizona's Cochise County to certify election results after a delay",
 '2020 Fall Sports',
 'Soccer',
 'Fire report 12-1-22',
 'Dems to make South Carolina, not Iowa, 1st voting state in 2024 presidential election season',
 'Dems move to make South Carolina, not Iowa, 1st voting state',
 'Fleetwood Mac singer-songwriter Christine McVie dies at 79',
 'Cal’s Bush wants affordable housing for marginalized people',
 'Fall Sports 2021',
 'Progress 2022',
 'Shopper 11/8/2022',
 'Thanksgiving traditions',
 'Cassville Democrat',
 'College Football Playoff expands to 12 teams in 2024 season',
 '20 Under 40',
 'LETTER TO THE EDITOR – Chillicothe News Constitution-Tribune',
 'Commission meeting minutes for Nov. 1 – Linn County Leader',
 'Deal finalized to divide Rams settlement money in St. Louis',
 'Veterans Day 2021',
 'Richmond volleyballers see ‘lot of successes’ in season-ending loss',
 'Police report 12-2-22',
 "Cal's Bush wants affordable housing for marginalized people",
 'Wentzville Updates',
 'Missouri Supreme Court won’t halt Kevin Johnson’s execution despite racial bias findings',
 'Ozark Adventures',
 'Graduation 2021',
 'Holiday Guide 2020',
 'New materials in Ozark Regional Library System',
 'GVMH Awarded Acute Stroke Ready Certification From The Joint Commission',
 'Jury begins deliberations in Harvey Weinstein rape trial',
 'Californianos tienen mayor acceso a atención médica con iniciativas de salud móviles y virtuales',
 'Veterans Day',
 'Community Calendar for Nov. 22 through Nov. 28',
 'HOLIDAY GUIDE 2921',
 "Cathi's Corner",
 'U.S. House Democrats elect Hakeem Jeffries to become first Black party leader',
 'Massive dust storm blows through NW Kansas',
 'Friday December, 2, 2022',
 'How to take action against antisemitism in your own life',
 'Death Notices: 11-17-22',
 'CEO works to ensure hospital’s long-term health',
 'Get Fit 2022',
 'Reform candidates lead in UAW races with 68% of vote counted',
 'Joyas ocultas en Honduras tienen ofertas para todo tipo de viajero',
 'R-I School Menu 11-10-22',
 'U.S. Soccer Star Christian Pulisic Mending, Hopes to Play in Saturday’s Match',
 'Obituaries: 11-17-22',
 'Biden signs legislation to avert rail shutdown',
 'Shopper 11/22/2022',
 'November 30, 2022 edition',
 'Fire report 12-2-22',
 'Biden sees economy avoiding recession, but risks remain',
 'Tools for Teachers presented by Weber Chevrolet',
 'After school programs work to close pandemic-driven achievement gaps for students of color',
 'Young hunters harvest 13,759 deer during early youth portion of firearms deer season',
 'Gas prices drop to levels not seen since February',
 'Why are so many people sick right now?',
 '316812525_479601164281704_1078839810498408052_n.jpg',
 'Speak Out 12-1-22',
 "U.S. Soccer star Christian Pulisic mending, hopes to play in Saturday's match",
 'Police report 12-1-22',
 'Show Me KC Schools’ SchoolAppKC launches for the fifth year',
 'Mahomes tabbed AFC Offensive Player of the Month',
 'Reform candidates lead in UAW races with 73% of vote counted',
 'R-I School Menu 11-17-22',
 'Bearcats rout Chiefs to finish the regular season undefeated',
 'Aline Kominsky-Crumb, underground cartoonist, dies at 74',
 '20 Under 40 2022',
 'Public Record',
 'Farmers, communities see need for U.S. 54 improvements',
 'ABMD Parade 2022',
 '3-3-22 Obituaries',
 'Tuesday, November 29, 2022',
 'Wednesday November, 30, 2022',
 'William Shatner, Giancarlo Esposito to make appearances at Planet Comicon KC',
 'Letter to the editor – Boonville Daily News',
 'EPA seeks to mandate more use of ethanol and other biofuels',
 'Friday, December 2, 2022',
 'Dr. Seuss themed “Festival of Lights” draws hundreds to downtown Cameron',
 'Hundreds receive anonymous letters targeting West County teenagers',
 'What is DME and why it matters if you have diabetes',
 'Jordan Martinook has hat trick Hurricanes beat Blues 6-4',
 'RAY COUNTY FAIR WRAPS, BUT MEMORIES REMAIN',
 'A rare recording of a musical by an 18-year-old Stephen Sondheim surfaces',
 'Lawmaker wants to use Missouri surplus to widen I-70',
 'Death Notices',
 'Commission meeting minutes for Nov. 15 – Linn County Leader',
 'Young Hunters Harvest 13, 759 Deer During Early Youth Portion Of Firearms Deer Season',
 '317502589_479601087615045_58795375827766158_n.jpg',
 'Progress 2021',
 'Jordan Martinook has hat trick, Hurricanes beat Blues 6-4',
 'Spring Sports 2022',
 'Missouri man admits 26-year Social Security fraud',
 'Land transfers',
 'Wednesday, November 30, 2022',
 'Shopper 11/1/2022',
 'Fall Bridal 2021',
 'Winter Sports 2021',
 'Letter to the Editor: – Boonville Daily News',
 'St. Francois State Park schedules bluegrass jam session for May 14',
 'Community Calendar for Nov. 15 through Nov. 21',
 'Kansas City World Cup leaders in Qatar as they prep for 2026',
 'Missouri’s poorest city may come as a surprise',
 'Graduation 2022',
 'Hermann Boys Basketball over Wright City. Go Bearcats!',
 'Winter Sports 2020',
 'Calendar Contest',
 'Tuesday, November 1, 2022',
 'GET FIT 2022',
 '12 best Elf on the Shelf accessories',
 'For many Hawaiians, lava flows are a time to honor, reflect',
 'Jonesburg Christmas parade returns for 12th year',
 "Here's how Missouri's U.S. House members voted to halt rail strike, grant sick leave for workers",
 'Get Fit 2021',
 'Flu season worsens as 44 states report high activity',
 'High court to rule on Biden student loan cancellation plan',
 'Aggies end regular season on a winning note',
 'Merging galaxies captured by James Webb telescope',
 'FALL SPORTS 2021',
 'Shopper 11/15/2022',
 'Community Calendar for Dec. 6 through Dec. 12',
 'Criticism mounts over proposed rule regarding age-appropriate material at public libraries',
 'Cassville Democrat',
 'Burrow tops Mahomes again, rallies Bengals past Chiefs 27-24',
 'The most common complaints filed to your local government',
 'Cass County escapees, accomplice charged with federal crimes',
 'Report: Classified items found at Trump storage unit in Fla.',
 'B-21 unveiled: Air Force unveils its newest stealth bomber',
 'George Clooney, Gladys Knight among Kennedy Center honorees',
 'Flash flood kills nine at church gathering in South Africa',
 'Noodle, the TikTok-famous ‘bones or no bones’ pug, has died',
 'Rangers score 3 in third period to rally past Blues 6-4',
 'Kirstie Alley, Emmy-winning ‘Cheers’ star, dies at 71',
 'The Truth About the Border',
 'Incoming GOP congressman says party won’t be ‘held hostage’ by McCarthy detractors',
 'Some Virginia players to get extra eligibility',
 'Lewis Capaldi: My mansion is a money pit!',
 "Jazzy 'Charlie Brown Christmas' swings on after 57 years",
 '‘Worth the wait’: U.S. Senate Democrats celebrate a 51-seat majority with Georgia win',
 "Pug who went viral on TikTok for 'no bones day' dies",
 'Emboldened athletes push back on old-school coaching methods',
 "Supreme Court weighs 'most important case' on democracy",
 'Warnock, Walker: Starkly different choices for Black voters',
 'Police report 12-4-22',
 'Firefighters respond to three-story building fire in St. Louis',
 'Experts worry expungements under Missouri marijuana law could be procedural ‘nightmare’',
 'Police report 12-7-22',
 'The 2022 word of the year has some scratching their heads',
 'Mississippi revises demands on Favre in welfare lawsuit',
 'US approves $4B sale of Abrams tanks to NATO ally Poland',
 'Harry and Meghan slam British tabloids in new Netflix series',
 'Supreme Court weighs ‘most important case’ on democracy',
 'Encore: Filmmaker Julia Reichert died Thursday at age 76',
 'Wednesday, December 7, 2022',
 'Saturday December, 3, 2022',
 'Reports: Cardinals sign catcher Willson Contreras to five-year deal',
 'US hits prominent Turkish executive with Iran oil sanctions',
 'Flu season worsens as 44 states report high activity',
 'End of public health emergency could cause spike in uninsured Missouri children',
 'Marijuana Now Legal In Missouri, But You Can’t Buy It Yet',
 'China eases anti-COVID measures following protests',
 'Five Ways To Safely Celebrate With Your Pet This Holiday Season',
 'Drake postpones New York shows for second time',
 'Kraft Heinz recalls ready-to-eat ham and cheese loaf products',
 'Man arrested in visitor’s fatal beating in New Orleans hotel',
 'Ravens sign Brett Hundley; Lamar Jackson unlikely Sunday',
 'Excelsior Springs man gets 40 years in prison for rape, sodomy',
 'Recreational marijuana becomes legal in Missouri today',
 '5 tips for donating your time or money this holiday season | PennyWise podcast',
 'Man arrested after egg allegedly thrown at King Charles III',
 'Rising deer population leads MDC to add more dates to hunting season in coming year',
 'Republican Senator files bill to address state’s high maternal mortality',
 'FBI warns of holiday scams: What to watch for, what steps to take',
 'Community Calendar for Dec. 4 through Dec. 10',
 'December 8, 2022 edition',
 'Lady Dragons outlast Plattsburg for first win',
 'Patrick Mahomes selected as Chiefs nominee for one NFL’s highest honors',
 'Iseah Jackson pleads guilty to involuntary manslaughter',
 'How To Support Veterans’ Families During The Holidays',
 'Slate of New Mexico regulatory candidates sparks concern',
 'Turkish strikes on US Kurd allies resonate in Ukraine war',
 'U2, Gladys Knight, George Clooney among 2022 Kennedy Center honorees',
 "Experts worry expungements under Missouri's marijuana law could be procedural ‘nightmare’",
 "Under China's 'zero COVID,' uncertainty reigns and unsettles",
 'US eliminated from World Cup after 3-1 loss to the Netherlands',
 "Chiefs look to avenge last season's playoff loss to Bengals",
 'Palestinians say killing caught on video was unjustified',
 'Republican Senator files bill to address Missouri’s high maternal mortality',
 'US intel chief thinking ‘optimistically’ for Ukraine forces',
 'Fire report 12-7-22',
 'Wheat pennies could bring you a pretty penny',
 'Two St. Louisans have big plans for a tiny house and an urban farm',
 'Shots fired into ceiling at fraternity party',
 'Gas prices \x97 How low could they go?',
 "Hockey's history shows handful of non-white pioneers",
 'Colorado Springs suspect formally charged with 305 counts in Club Q shooting',
 "Voice of Missouri's 'Smokey Bear' inducted into Conservation Hall of Fame",
 'How to breathe new life into your home without a complete renovation',
 'New study: Millennial parents struggle to enjoy fun moments with their kids',
 "Mollie King admits getting whole family 'fully dressed' is an achievement",
 'Offset only wants fans to share positive memories of Takeoff',
 'Why Missouri marijuana users will still need a card to buy Dec. 8',
 'Chiefs QB Patrick Mahomes unfazed by foot issue',
 'FBI arrests Nixa man on bank robbery charge',
 'Keep COVID military vaccine mandate, defense chief says',
 "Just Being Healthy Might Prevent Many Cases of Crohn's, Colitis",
 'Biden releasing nearly $36B to aid pensions of union workers',
 'Wednesday December, 7, 2022',
 'Listen to Across the Sky podcast; episode 34 coming soon!',
 "How 'goblin mode' became Oxford's word of the year",
 'Trump Organization found guilty of tax fraud',
 'Bribing Folks Can Help Them Meet Weight-Loss Goals, Study Finds',
 'Kansas woman found dead after boyfriend’s arrest in North Carolina',
 'Search for missing man continues 9 months after disappearance',
 'Georgia a nearly unanimous No. 1 in AP Top 25',
 'Money worries making the holidays less merry? 5 tips from a financial therapist to rein in financial anxiety',
 'UNLV hires former Missouri coach Barry Odom to head program',
 'Chapter 1: Walked into thin air | Late Edition: Crime Beat Chronicles podcast',
 'Police report 12-6-22',
 'Missouri Democrats re-elect party chair following online feud over leadership',
 'Broncos WR Courtland Sutton unlikely against Chiefs',
 'Griner for Bout: WNBA star freed in US-Russia prisoner swap',
 'Arrest made after wrong-way pursuit on Highway 63',
 'Around Linn County – Linn County Leader',
 'Warrant: Texas FedEx driver says he strangled girl in van',
 "49ers' Super Bowl odds skyrocket minus Jimmy Garoppolo",
 'Missouri Secretary of State looks to control public library policies on age-appropriate books',
 'Apple: Most iCloud data can now be end-to-end encrypted',
 'Saturday, December 3, 2022',
 'Trump Organization convicted in executive tax dodge scheme',
 'Democratic Sen. Warnock wins Georgia runoff against Walker',
 'House passes defense bill scrapping COVID vaccine mandate',
 'December 7, 2022 edition',
 'Police report 12-8-22',
 "The latest on Iran's protests, morality police and a silent majority",
 'Pfizer seeks FDA authorization for omicron vaccine in kids under 5',
 'House January 6 committee has decided to make criminal referrals, chairman says',
 'Study finds Taco Bell as most popular Missouri fast food option',
 'Tuesday, December 6, 2022',
 "Marijuana now legal in Missouri, but you can't buy it yet",
 'Family of Black refugee Patrick Lyoya sues Grand Rapids police officer for fatal shooting',
 'Vandalism at Missouri elementary school includes a swastika',
 "Billie Piper: 'Fame is poisonous and depressing'",
 "'When Christmas Was Young' adds extra ingredient to holiday film formula",
 "Man who shot Lady Gaga's dog walker gets 21 years in prison",
 "Noodle, the TikTok-famous 'bones or no bones' pug, has died",
 "Peru's president arrested after attempting to dissolve congress",
 "What's the secret sauce that lets some poor countries prosper?",
 'Marijuana now legal in Missouri, but you can’t buy it yet',
 'Group aiding kin of slain CIA officers comes out of shadows',
 'The truth about the border',
 "CDC Director Rochelle Walensky advises on the 'tripledemic'",
 '5 tips for safe online shopping this holiday season',
 "Molten lava on Hawaii's Big Island could block main highway",
 'Florida QB Anthony Richardson declares for NFL draft',
 'Heart attacks more prevalent over holidays: How to stay safe',
 'Suspect fatally shot, 2 found dead in Missouri home',
 "Man arrested in visitor's fatal beating in New Orleans hotel",
 'Conservation efforts have brought back an endangered species of fish in the Amazon',
 'UM System keeps marijuana prohibited on all grounds',
 'Am I not here, I who am your mother?',
 'How to layer clothes for winter',
 'Pelé’s daughters say he was hospitalized last week with a lung infection following bout of Covid-19',
 "Iran morality police status unclear after 'closure' comment",
 'Pug who went viral on TikTok for ‘no bones day’ dies',
 'Kansas high schools investigate offensive chants, taunts at game',
 '‘Spread a little holiday cheer’: Officers hand out $100 bills',
 'Fire report 12-8-22',
 "Kirstie Alley, Emmy-winning 'Cheers' star, dies at 71",
 'Trump rebuked for call to suspend Constitution over election',
 'Holiday Tech Gift Ideas for Everyone on Your Shopping List',
 'Dem governor: Caring is key to connecting with rural voters',
 'Women sue Musk’s Twitter alleging discriminatory layoffs',
 "Oxford Dictionaries names 'goblin mode' its word of the year",
 'Missouri violated collective bargaining rights of prison guards, appeals court rules',
 'Speak Out 12-6-22',
 '25 richest people in America and how they did it',
 'Renowned boxing referee Mills Lane dies at 85',
 'Both sides see high stakes in gay rights Supreme Court case',
 'Fire report 12-6-22',
 'Report: Jim Leonhard resumes assistant role at Wisconsin',
 "US intel chief thinking 'optimistically' for Ukraine forces",
 'FTC sues to block Microsoft-Activision Blizzard $69B merger',
 '‘If your dog died ... prove it to us’: Olive Garden manager fired after time-off rant',
 'After World Cup, US men recede to background for 3 1/2 years',
 'Tuesday December, 6, 2022',
 'Hawaii remembrance draws handful of Pearl Harbor survivors',
 'DHS delays REAL ID deadline another two years',
 'Speak Out 12-4-22',
 'Lawmaker wants to use Missouri surplus to widen I-70',
 'Jazzy ‘Charlie Brown Christmas’ swings on after 57 years',
 'Missouri football to play Wake Forest in Gasparilla Bowl',
 'Marijuana expungement petitions open Thursday amid confusion',
 'U.S. Cities With the Largest Increase in Fair Market Rent Since Pre-COVID',
 'Kansas man wanted in Nebraska woman’s disappearance arrested in Belize',
 'Labor Market Kept It Tight in 2022',
 'Messi scores, Argentina beats Australia 2-1 at World Cup',
 'Calendar Contest',
 'Who is Missouri’s new attorney general, what are his priorities?',
 "Jesuit artist has ministry cut; Vatican doesn't prosecute",
 'Warnock or Walker? Georgia runoff to settle last Senate seat',
 'Fire report 12-4-22',
 'Chiefs look to avenge last season’s playoff loss to Bengals',
 'Justices spar in latest clash of religion and gay rights',
 "Volodymyr Zelensky is TIME Magazine's 2022 Person of the Year",
 'Ho-ho-hold the holiday decorations',
 'Chiefs’ Kadarius Toney, Joe Thuney questionable vs. Broncos',
 "'I want to talk': Griner opened up during her long trip home",
 '‘Historic moment’: Lamar Johnson wrongful conviction case goes before a St. Louis judge',
 "It's Snow Season: Stay Safe on the Slopes",
 'Reports: Jason Garrett a finalist for Stanford opening',
 'Reporter Lloyd Newman, known for Ghetto Life 101, died this week at age 43',
 'Heading to the cabin this winter? Get these cozy gifts for the skier in your life',
 'US lawmakers introduce legislation to ban TikTok',
 'FBI increases reward in hunt for Cass County escaped inmates',
 'School construction update: All’s well, so far',
 'EXPLAINER: Why fusion could be a clean-energy breakthrough',
 "Golden Globe noms led by 'Banshees', 'Everything Everywhere'",
 'Possible inspiration behind Patrick Mahomes no-look pass',
 'Steelers draw Baltimore backup again, chase five consecutive wins over rival',
 'Wednesday December, 14, 2022',
 'Warrant: Texas FedEx driver says he strangled girl in van',
 'The U.S. has taken custody of the alleged bomb maker in the 1988 Lockerbie attack',
 "Matt Goss can't be silly in public",
 'Missouri DESE votes to recommend teacher pay increase',
 'Tuesday, December 13, 2022',
 'Democratic Sen. Kyrsten Sinema switches to independent',
 'Rone ready to leave legislative role behind',
 "Mahomes' brilliance for KC blighted by more turnover trouble",
 '3 dead in Louisiana as US storm spawns Southern tornadoes',
 'Amazon answers questions about safety changes year after deadly warehouse tornado',
 'Facing COVID surge, China expanding hospitals, ICUs',
 'Review: ‘Avatar: The Way of Water’ is a big screen blast',
 'Friday December, 9, 2022',
 'Bipartisan group of lawmakers push to restrict foreign ownership of Missouri farmland',
 'Last-minute tech gifts worth waiting for, according to BestReviews expert',
 "Pink Card tells the story of Iranian women's fight for freedom, rooted in soccer",
 'Bengals safety fined for faking injury vs. Chiefs',
 'Calendar Contest',
 'Iseah Jackson sentenced to seven years for deaths of woman, 11-year-old girl',
 "Prince Harry says Nazi outfit 'was one of his biggest mistakes'",
 'Police Report',
 'Argentina beats Croatia in the first semi-final of the World Cup',
 "Nelly Cheboi, who creates computer labs for Kenyan schoolchildren, is CNN's Hero of the Year",
 'Hospitalizations signal rising COVID-19 risk for US seniors',
 'Ram recalls 1.4 million trucks; tailgates can open unexpectedly',
 'Morocco reaches World Cup semifinals, tops Portugal, Ronaldo',
 'Health experts encourage mask-wearing as respiratory illnesses rise this winter',
 'Kari Lake challenges her defeat in Arizona governor’s race',
 'Customer shoots KFC employee because store ran out of corn, police say',
 'NFL fines Bengals safety for faking injury',
 "Library doesn't budge in latest union negotiations",
 'About 300 Lincoln University workers to get one-time ‘retention payments’',
 'Chiefs WR Juju Smith-Schuster aiding families in need',
 'Bill protecting same-sex, interracial unions clears Congress',
 'Chiefs survive Broncos comeback to win 14th straight over Denver',
 'The fight against Judeo-Christian values',
 "Majority of Americans don't want Biden to run again",
 'Memorial to American writer Wahl placed at World Cup game',
 'Ashland police provide update on case of missing teen',
 'Mauna Loa lava no longer imminent threat to Hawaii highway',
 "Lamar Johnson's wrongful conviction case goes before a St. Louis judge: ‘Historic moment’",
 'Saturday December, 10, 2022',
 'Chiefs downgrade Toney to out for Broncos game',
 'KU hammers Mizzou by 28 as Border War returns to Columbia',
 'St. Louis named among Top 10 metros for first-time homebuyers',
 '3 bald eagles die, 10 sick after eating euthanized animals',
 'MDC reports 16,032 deer harvested during antlerless portion',
 "AHA News: Kentucky Wildcats Basketball Player Won't Be Sidelined by Heart Surgery",
 'Public Record',
 'Traffic impacts expected for Mizzou rivalry matchup against Kansas',
 'Canada is expanding categories for medically assisted death',
 'Philadelphia ordered to remove box covering Columbus statue',
 'Facing COVID-19 surge, China expanding hospitals, ICUs',
 "Judge formally dismisses Trump's case for a Mar-a-Lago special master",
 'KC mayor slams Independence 4-day school week as ‘gimmick,’ says it will hurt parents and kids',
 '6 unique gifts to give this Christmas',
 'OATS Transit Board elects officers for 2023-2024',
 'Soccer writer Grant Wahl dies at World Cup match in Qatar',
 '2 plead to misdemeanors in Missouri boarding school case',
 "James Corden apologized for using Ricky Gervais' joke",
 'Hollister School Board recognizes monthly teacher, staff member',
 "Selma Blair's MS diagnosis 'has damaged her career'",
 'Aaron Taylor-Johnson ‘has filmed iconic 007 gunshot opening’',
 'Children’s & Family Creative Arts Emmy Award Winners Announced: See the Full List',
 'Supreme Court won’t block California flavored tobacco ban',
 'Tina Turner’s cancer survivor son Ronnie dies aged 62 ‘after struggling to breathe’',
 'December 10, 2022 edition',
 'Whiteman runway damaged after B-2 makes emergency landing',
 'Suspected Russian FSB officer charged in U.S. sanctions case',
 "Marijuana now legal in Missouri, but you can't buy it yet",
 'Mars rover captures 1st sound of dust devil on red planet',
 "Browns' Amari Cooper misses practice, questionable vs. Bengals",
 'Florida Gov. DeSantis seeks grand jury investigation of COVID-19 vaccines',
 'FBI releases descriptions as hunt for Cass County escaped inmates continues',
 'James McAvoy planning to step back from acting',
 "Marijuana now legal in Missouri but you can't buy it yet",
 'Bomb threat disrupts Patti LaBelle concert in Wisconsin',
 'Shoppers, workers clash over post-pandemic expectations',
 'Trump probe subpoena served on Georgia secretary of state',
 'Raft with US flag caught in plain view off Havana coast',
 'Biden aims to narrow trust gap with US-Africa leaders summit',
 'UN chief: Syrians face dire winter if aid from Turkey is cut',
 'Long COVID contributed to about 3,500 deaths, CDC says',
 'PUBLIC ANNOUNCEMENT',
 "New this week: 'Banshees', Whitney Houston, 'High on Life'",
 'Kansas man charged in 1980 killing of nursing student',
 'Griner in ‘good spirits’ in US after Russian prisoner swap',
 'Wednesday, December 14, 2022',
 'Clemson DE Myles Murphy entering NFL draft',
 'First $100K prize won in Red Hot Cash Missouri Lottery game',
 'Have young Americans lost their sense of patriotism?',
 "SpaceX gives rival's internet satellites ride to orbit",
 'Police report 12-11-22',
 'Battling High Blood Pressure? Adding Yoga to Your Workout Might Help',
 'Walmart customers complain about double charges; retailer responds',
 'Land Transfers 12-08-22',
 '3 gift ideas for every kid on your list',
 'WATCH: Mahomes, McKinnon connect on flip touchdown pass',
 'Sandy Turner: Second time around is less stressful',
 'Another Big Bank Gets Into Small-Dollar Loans',
 'New plaque honors Missouri for preserving history of ‘Mighty Mo’ battleship',
 'Streets of Argentina turn into party as team reaches final',
 'The Locker Room: new owners, new chapter',
 'Mississippi State football coach Mike Leach dies at 61',
 'Boone County Sheriff’s Office continues efforts to fill positions during staffing shortage',
 'American author Michael Shellenberger releases ‘Twitter Files Part 4’',
 'Shortages for key drugs continue amid surge in illnesses',
 "In a post-Roe Missouri, lawmakers turn focus on new mothers' health and child care",
 'Missouri Republicans renew attempts to restrict transgender sports, health care, education',
 'Which digital camera is best for beginners?',
 'Duke of Sussex fears he blocked out memories of Princess Diana',
 'How to Safely Dispose of Used Medical Sharps During Holiday Travel',
 'Joseph Elledge: ’48 Hours’ features Missouri murder',
 'Community Calendar for Dec. 13 through Dec. 19',
 'Pioneering Black feminist Dorothy Pitman Hughes dies at 84',
 'Year after deadly tornado, families question safety at Amazon',
 '‘I want to talk’: Griner opened up during her long trip home',
 'Longtime Agape director testifies in Missouri court to stay off child abuse registry',
 'Prevent missing out on activities: Get your flu vaccine today',
 'Morocco marches on - eliminating Portugal in the quarterfinals of the World Cup',
 'Stock up on COVID kits (on sale now) before spending time with family this holiday season',
 'Police report 12-13-22',
 "Review: 'Avatar: The Way of Water' is a big screen blast",
 'Seitz files bills for 2023 legislative session',
 '13 great gifts for people with green thumbs',
 'CVS, Walgreens finalize $10B in settlements over opioids',
 'Scott County sheriff says he will seek lawful return of K-9 Rex',
 'Cassville Democrat',
 'USC QB Caleb Williams wins 2022 Heisman Trophy',
 "How Kabrina Clark's wish came true: A tropical surprise",
 'US sending $275 million in military aid to Ukraine',
 'Community Calendar for Dec. 11 through Dec. 17',
 'Ravens plan for QB Tyler Huntley to start vs. Steelers',
 'Ameren Missouri launches program to help customers return to permanent housing',
 'The AP Interview: Hutchinson says Trump worst choice for GOP',
 'Ecstatic Moroccans celebrate World Cup victory over Portugal',
 'US postage stamp to honor civil rights icon John Lewis',
 'No more Tiger Kings, cub-petting: Big Cat Public Safety Act passes in U.S. Senate',
 'Saint Francis to host annual Live Nativity in Cape Giradeau',
 'Tracking early morning showers and a cool Saturday',
 'Cher’s mum dead aged 96 after pneumonia battle',
 '3 ways women are taking control of their finances to reenergize their future',
 "The rise of extremism in Germany's military",
 'Kobach to nominate ex-primary rival Mattivi as KBI director',
 'Missouri Republicans file bills to automate expungements for non-violent offenses',
 'A derecho swept through the Midwest a year ago. Now there are efforts to better define the storms',
 'Police report 12-14-22',
 'US blizzard expected to impact millions in Rockies, Midwest',
 'MSU-WP students experience democracy in action during Nov. 8 election',
 "Supreme Court won't block California flavored tobacco ban",
 'Best beauty deals for the holiday shopper',
 'The Most Risky Industries to Start a Business',
 "Director Ron Howard discusses 'Thirteen Lives' and his new memoir 'The Boys' | Streamed & Screened podcast",
 'December 14, 2022 edition',
 '5 winter travel tips to save your patience and your pocketbook | PennyWise podcast',
 'Great Ways to Encourage Imagination During Playtime',
 "A new law in Russia is Putin's latest attack on LGBTQ rights",
 'CPD heads to the end of 2022 with staffing shortages unresolved',
 'Neil Young working on multiple unfinished records',
 'North Texas hires Eric Morris as head coach',
 'Tracking dry conditions early this week, more rain on Tuesday',
 "Boone County's new prosecutor targets drug crimes, commits to better communication",
 'Columbia man sentenced to seven years in prison for manslaughter charges',
 'Report: Ravens QB Lamar Jackson could return in Week 16',
 'Confederate monument set to be removed from Virginia capital',
 'Columbia STEM Alliance Math Mentors work to reduce achievement gap',
 'France advances to semifinals at World Cup, tops England 2-1',
 '15 unconventional Christmas albums from the past 50 years',
 "UC's academic workers strike brings stress to undergraduates",
 'John Pace',
 'Aurora School District faces two lawsuits claiming children sexually assaulted on bus',
 "Peru president's power grab recalls country's dark past",
 'Marijuana now legal in Missouri, but you can’t buy it yet',
 'Devotion to Virgin Mary draws millions to Mexico City shrine',
 'Federal Trade Commission sues Microsoft to block Activision Blizzard deal',
 'Police report 12-9-22',
 '‘The Stranglers’ drummer Jet Black dead aged 84 after years of illness',
 'Ex-cop who kneeled on George Floyd’s back gets 3.5-year term',
 'Missouri federal court postpones Amber Waterman trial',
 'Massive US storm brings tornadoes to South, blizzard threat',
 "Tina Turner's son Ronnie died following cancer battle",
 "Choi condemns MU student's racist social media post",
 "Meghan Markle claims Thomas Markle's phone was 'compromised'",
 'Titans WR Treylon Burks (concussion) out vs. Jaguars',
 'Retiring U.S. Sen. Roy Blunt to speak to SEMO graduates this weekend',
 'Missouri has roughly $150 million left to distribute in federal housing assistance',
 'Tuesday December, 13, 2022',
 'Julian Lennon: Yesterday was ruined for me by weird John Lennon scene',
 'Raiders at Rams: TNF Preview, Prop Picks, Prediction',
 'Tensions run high in north Kosovo as Serbs block roads',
 'Bengals, Joe Burrow scheming for first win vs. rival Browns',
 'Land transfers',
 'Are radiant heaters safe for your home?',
 "Prosecutor: Oath Keepers saw Jan. 6 as 'first battle' in war",
 'EU reaches deal on Ukraine aid, tax on big corporations',
 'Boone County murder suspect appears in court Tuesday',
 'Company starting to recover oil from Kansas pipeline spill',
 'Brittney Griner arrives in the US after being released from Russian custody in a prisoner exchange',
 'A man ran 415 circles around a roundabout, a 36-mile ultramarathon',
 'Listen to Across the Sky podcast; episode 35 coming soon!',
 "Report: Tom Brady considering 'all options' for 2023",
 'Saturday, December 10, 2022',
 'Missouri Southern on probation for football violations',
 "Golden Globe noms led by 'Banshees,' 'Everything Everywhere'",
 'Viktor Bout is back in Moscow. Is he still a national security threat to the U.S.?',
 'Nicolas Cage used to think he was an alien',
 "New this week: 'Banshees,' Whitney Houston, 'High on Life'",
 'Top tech tips: How to create the best interactive virtual experiences',
 'CDC urges masking return as tripledemic surges',
 'Friday, December 9, 2022',
 "McCarthy's race for speaker risks upending House on Day One",
 'Kansas City Chiefs can clinch AFC West, trip to playoffs this Sunday',
 'Best gift for record-player enthusiasts',
 'These holiday plants could harm your pets',
 'Air Force grounds entire B-2 fleet after emergency landing',
 'FTX founder agrees to extradition to US, attorney says',
 'Missouri brothers catch largest raccoon in state history',
 'AP PHOTOS: World Cup highlights from Day 24',
 'Panel warns FDA’s beleaguered tobacco unit lacks direction',
 'Nobel laureate economist faces sex harassment investigation',
 'Up-and-coming digital creators featured on Times Square billboard',
 'US company to send team to look into Berlin aquarium rupture',
 "Ex-Mafia boss 'Cadillac Frank' Salemme dies in prison at 89",
 'axlyn 4.jpeg',
 'Dan McLaughlin ‘stepping away’ from Cardinals broadcasts after third DWI charge',
 'Police: Suburban Kansas City officer kills armed suspect',
 'Wichita man arrested for abducting child in Arkansas',
 'Shop St. Joseph winning numbers announced',
 'Officer, suspect die, trooper wounded in Arkansas shootings',
 'E.W. Scripps announces formation of new sports division',
 "An 84-year-old filmmaker looks into a donkey's soul in 'EO'",
 '‘Avatar 2’ makes waves with $134 million domestic debut',
 'Brittany Murphy’s brother convinced she was murdered',
 'Snow pummels parts of Northeast; thousands without power',
 'Police report 12-15-22',
 'Dead boy pulled from rubble of latest Russian hit on Ukraine',
 'Two people killed in Clay County crash',
 "What to watch as Jan. 6 panel cites Trump's 'attempted coup'",
 'axlyn 7.jpeg',
 'Four injured in area vehicle crashes',
 'Prosecutor: Arkansas police recruit’s death was accidental',
 'Dozens feared buried after landslide hits Malaysian campsite',
 'Kangaroo spotted in Jefferson County is safe',
 'Robotic pets offered free to independent seniors Robotic pets offered free to independent seniors',
 'Highland Park suspected shooter’s father is charged with felony reckless conduct',
 'Biden administration proposes crackdown on scam Medicare ads',
 'Argentina Wins 2022 World Cup After Battling France in Extra Time',
 'Saturday, December 17, 2022',
 'Governments move closer to deal at biodiversity conference',
 '1 wounded, suspect dead after Amazon warehouse shooting in Arizona',
 'Twitter bans linking to Facebook, Instagram, other rivals',
 'McCarthy’s race for speaker risks upending House on Day One',
 'Chiefs Patrick Mahomes to invest in Major League Pickleball franchise',
 'Lawmakers quick to unload FTX founder’s contributions',
 "Joplin sex offender's registration violation charge dismissed",
 'OSHA fines Silver Dollar City over $14,000 after park employee death',
 'Community reacts to appointment of new Kansas City police chief',
 'Hidden gems of Amazon that will solve your holiday shopping',
 "Ellen DeGeneres 'heartbroken' over death of Stephen 'tWitch' Boss",
 'What’s in the House January 6 committee report summary',
 'Funeral services to be held for Salisbury woman killed in fire',
 'US storm brings tornadoes, blizzard-like conditions; 2 dead',
 'Lawmaker targets gas station gambling in Missouri',
 '4 chimps loose in monkey house in Swedish zoo, 3 shot dead',
 'Pakistani Taliban overpower guards, seize police center',
 '$698M deal to end Monsanto PCB pollution lawsuit in Oregon',
 "Meghan Markle’s wedding dress designer says it had to be 'flawless'",
 'American student missing in France is alive, father says',
 'Apple TV+ Cancels ‘Shantaram’ After Season 1',
 "AHA News: Why a Sportswriter's Sudden Death Should Lead You to Ask About Your Own Family History",
 'Community Calendar',
 'Retail sales drop at start of key holiday shopping season',
 'Chiefs could soon have WRs Toney, Hardman back',
 'Fire report 12-18-22',
 "Dan McLaughlin is out as Cardinals broadcaster by 'mutual decision'",
 'Contested natural gas pipeline granted permanent certificate',
 'Police report 12-16-22',
 'Planned Roadwork for Northwest Missouri, December 19-25',
 "Take the plunge: Avatar's underwater scenes are immersive and extraordinary",
 'Saturday December, 17, 2022',
 'US court rejects maintaining COVID-19 asylum restrictions',
 'Journalist suspensions widen rift between Twitter and media',
 'Brazilian singer Anitta has her sights set on global stardom',
 'The best gifts you can buy on Amazon under $100 today',
 'Pastor’s Pen 12-1-22',
 "Gholston's buzzer-beating 3 gives MU 68-66 win over UCF",
 'Bond set at $50K for father of July 4 shooting suspect',
 "'Something to Talk About' songwriter Shirley Eikhard dies",
 "Janelle Monáe, the not-so-secret weapon of 'Glass Onion'",
 'Janelle Monáe, the not-so-secret weapon of ‘Glass Onion’',
 'Robotic pets offered free to independent seniors',
 "Brittney Griner says she'll play in the upcoming WNBA season",
 'Elon Musk suspends — then restores — the Twitter accounts of several journalists',
 'Two juveniles killed in fire at Columbia apartment complex',
 '‘Something to Talk About’ songwriter Shirley Eikhard dies',
 'Father of July 4 shooting suspect charged with 7 felonies',
 'Winning Shop St. Joseph numbers announced',
 'City of Columbia preparing for incoming winter weather',
 'Rail union leader retires to clear way for upstart who won',
 '‘We will not rest’ : Search continues for elderly mid-Missouri woman',
 'Anti-abortion priest Pavone defrocked for blasphemous posts',
 'How to save the most on Apple, Samsung, Google and other best-brand products this season',
 'US storm brings tornadoes, blizzard-like conditions; 3 dead',
 'Push to repeal California anti-oil law inches closer to goal',
 'Bre Faircloth named new Royal Heights principal',
 'Best noise-canceling headphones for travel',
 'First Baptist Church Free Community Christmas Dinner',
 "Gholston's buzzer-beating 3 gives MU a 68-66 win over UCF",
 "Miriam Margolyes will 'have a word' with Leonardo DiCaprio",
 'Trump’s digital cards sell out within a day',
 'Officials: Colorado firm’s alfalfa cubes may kill horses',
 'Transgender inmate scheduled for Jan. 3 execution asks Missouri governor for mercy',
 'Man wins $25K for life lottery prize … six times',
 'December 15, 2022 edition',
 'Choi condemns racist social media post; Black students say more action needed',
 'Kelce moves into fourth all-time on tight end receiving yards list',
 'TSA raising fines after finding record guns in carry-ons',
 'Jamie Spears: Britney could have died without conservatorship',
 '‘We will not rest’: Search continues for elderly mid-Missouri woman',
 'BookTrib’s Bites: Exciting Stories for the Holidays',
 'December 17, 2022 edition',
 'Dame Sheila Hancock slams trend for public emotion',
 'Hanukkah starts Sunday: These 10 menorahs will help light up all 8 nights',
 'What to watch as Jan. 6 panel cites Trump’s ‘attempted coup’',
 'Iran authorities arrest actress of Oscar-winning movie',
 'Famed LA cougar P-22 euthanized following health problems',
 'OSHA fines Silver Dollar City after the death of a maintenance employee this summer',
 'Kansas City charity helping victims of deadly Columbia apartment fire',
 'Connecting generations: Final tournament held on Jefferson Middle basketball court',
 'Report: Packers release veteran WR Sammy Watkins',
 'Biden administration proposes phasing out fluorescent bulbs',
 'Beyonce sent message of support to Meghan, Duchess of Sussex',
 'Woman injured in London concert crowd crush dies in hospital',
 'Arkansas deputy killed during Wreaths Across America escort',
 'Inside the Black Box: Learn more about the role race plays in entertainment',
 'Wife: Journalist Wahl died of aortic aneurysm at World Cup',
 "What's in the House January 6 committee report summary",
 "Peru's dark past surfaces as young protester is laid to rest",
 'Players provide perspective on Royals downtown stadium',
 'Ex-Mafia boss ‘Cadillac Frank’ Salemme dies in prison at 89',
 'Cities With the Most Working Teens',
 'Police report 12-18-22',
 'Man attempting U-turn fatally shoots himself in groin',
 'Posts on coming cemetery décor enforcement raise concerns in Liberty',
 'Iron County man accused of setting fires at national park, pointing crossbow at forest service officer indicted',
 'Messi carries the weight of Argentina into World Cup final',
 'Amazon releases new Fire HD 8 tablets with Disney Design bundle',
 'Roger Stone says he saw demon portal open over White House',
 'William Woods names new Director of Athletics',
 'Ozark, Mo. teenager charged with murder after girl found dead with fentanyl in her system',
 "Lawmakers quick to unload FTX founder's contributions",
 'Company reopens most of pipeline following Kansas oil spill',
 'Lewis Capaldi wants new single Pointless to become a wedding song',
 'Morgan County man faces child porn charges',
 'Table Rock Lake Chamber of Commerce wins awards',
 'Broncos rule out Russell Wilson for Cardinals game Sunday',
 'Harry and Meghan want an apology from the Royal Family',
 'NFL fines Broncos WR for contacting official',
 "Officials: Colorado firm's alfalfa cubes may kill horses",
 'Suspect dead, 2 police officers shot and killed at motel in Bay St. Louis, Mississippi',
 'Giants, Commanders clash in crucial divisional rematch',
 'Fire report 12-16-22',
 'USC adds Pac-12 receiving yards leader Dorian Singer',
 "Movie Review: 'Avatar: The Way of Water'",
 'How to Financially Coexist With Boomerang Kids',
 'California lowers incentives for rooftop solar panels',
 'Holiday helper: 5 tips to help manage your spending this holiday season',
 'Jewish festival of lights begins in Ukraine as battles rage',
 'Chiefs grind out win in Houston, clinch AFC West title',
 'U.S. Supreme Court extends border restrictions just before they were set to end',
 'Keep safety in mind when donating food',
 'Amazon’s plastic waste increased by 18 percent in 2021',
 'Best tech gift even if it arrives late',
 'Gov. Abbott, border states prepare for deluge of crossings with Title 42 ending',
 "Prosecutor: Arkansas police recruit's death was accidental",
 '‘The Best Man’: Is ‘The Final Chapters’ Really the End?',
 'MDC reports three elk harvested during 2022 season',
 'The largest lottery jackpots in US history',
 'Frankincense, myrrh: Ancient scents of season',
 'Twitter moves to reinstate some journalists’ suspended accounts',
 'Prosecutor in Lamar Johnson case defends his actions',
 'Northwest 56, Emporia 64',
 'Seasonal Cocktails from St-Rémy Signature',
 'Micrometeorite possibly behind Russian space capsule leak',
 'Duquesne man takes plea offer in street robbery case',
 'Nixa police warn of phone scammers impersonating department',
 'Part of East Walnut Street closed due to Guitar Building structure issues',
 'Northwest 65, Emporia 74',
 'The best gifts you can buy on Amazon under $50 today',
 'Pastor’s Pen 10-27-22',
 '3 dead in Louisiana as US storm spawns Southern tornadoes',
 'Missouri awards over $40 million in low-income housing tax credits',
 'Pope wrote resignation note in case of health impediment',
 'Doolittle firefighters receive Birthday Blessings from community',
 "'The Taliban took our last hope': College education is banned for women in Afghanistan",
 'Chiefs’ Andy Reid, Patrick Mahomes share stories about late coach Mike Leach',
 'Senate passes defense bill rescinding COVID vaccine mandate',
 'MDC reports 16,032 deer harvested during antlerless portion',
 'Three teens hurt in one-vehicle crash in Miller County',
 'These 10 top deals on Amazon will help you finish your holiday shopping',
 'Missouri Republican pushes for constitutional change to recognize same-sex marriages',
 'Tech gifts that will actually make life easier',
 'KCI officer shot Friday morning at Kansas City Airport',
 'Family begs for help after Missouri teen vanishes',
 'Best Funko Pop toy gifts for collectors',
 'Santa answers the tough questions kids ask at Christmas',
 '2-year-old survives being swallowed, spit out by hippo, police say',
 '3 escaped chimpanzees recaptured in Sweden; 4 shot dead',
 "Final Exams Don't Have to Be High Stress for Your Teen",
 'Vice President Harris looks back at 2022, and ahead to 2023',
 'A member of the Jan. 6 committee on its final hearing',
 '3 product labels help you do good while shopping',
 'Trump referral: What laws the Jan. 6 panel says were broken',
 "Gholston's clutch 3-pointer lifts Missouri over UCF 68-66",
 'Working at 76: Inflation forces hard choice for older adults',
 '11 people seriously injured amid turbulence on Hawaii flight',
 'Senate passes defense bill rescinding COVID-19 vaccine mandate',
 'Combo of COVID Plus Flu Can Bring Severe Illness to Kids',
 "DONNA ERICKSON'S BEST BITES: Bake Swedish pepparkakor cookies for holidays",
 'Bessie Mae Kelley is one of the earliest known women to hand-draw animated films',
 'Jan. 6 panel urges Trump prosecution with criminal referral',
 'Twitter suspends journalists who wrote about Elon Musk',
 'Homicide detective in Lamar Johnson’s case says he had no evidence',
 'TSA finds record number of guns on passengers at checkpoints',
 'Harvey Weinstein found guilty of rape in Los Angeles trial',
 'Jan. 6 panel urges ethics review for McCarthy, GOP lawmakers',
 'Lamar Johnson denies role in killing that led to life term',
 'Ravens QB Tyler Huntley clears concussion protocol',
 'Title 42 to remain in place for now as Chief Justice John Roberts temporarily freezes order meant to end it',
 'Missouri top lawmaker reacts to Royals’ proposal for downtown stadium',
 'Missouri Chamber of Commerce to help childcare facilities following hundreds of closures',
 'How people ordered meals online in 2022, according to DoorDash',
 'Ukraine: Russian strikes thwarted, wreckage hits buildings',
 'Two face drug charges in Scott City',
 'France celebrates WCup victory, fans cheer Morocco team',
 'Officer injured after chase ends in crash, KCI airport officer shot placing suspect in custody',
 'Forgot to buy a present? 3 tips to simplify last-minute gift-giving',
 'Five tips for new EV drivers',
 'Fire report 12-15-22',
 "An 84-year-old filmmaker looks into a donkey's soul in ‘EO’",
 'Gholston’s clutch 3-pointer lifts Missouri over UCF 68-66',
 'Air of familiarity underscores Raiders-Patriots matchup',
 'Calendar Contest',
 'Missouri governor taps political newcomer to serve as next state Treasurer',
 'Información útil: 5 consejos para administrar tus gastos en esta temporada de fiestas',
 "Trump's taxes: House OKs bill on IRS audits of presidents",
 'Lottie Moss ‘traumatised’ after being mocked on modelling jobs for eating',
 'Bethlehem rebounds from pandemic, lifting Christmas spirits',
 'QB Jayden Daniels announces return to LSU',
 'Weather Updates | Blizzard warning in effect in Montana',
 'Congress moves to ban TikTok from US government devices',
 'Police investigate artillery shell found at Branson, Mo. wilderness park',
 'Water pressure ‘fluctuating’ in Jackson amid frigid weather',
 'US: Jan. 6 participant arrested after California standoff',
 'Missouri Gov. Parson selects next state treasurer',
 'South Africa marks holidays despite nationwide power cuts',
 'December 22, 2022 edition',
 'Friday December, 23, 2022',
 'High court temporarily blocks lifting of asylum restrictions',
 'Man drowns on frigid day at Lake of the Ozarks',
 "A group's work to help migrants who were sent to Kamala Harris' home on Christmas Eve",
 'Missouri AG office has taken outsized role in opposing wrongful conviction cases',
 'Violent California earthquake damages homes, disrupts power',
 'NASA Mars lander InSight falls silent after 4 years',
 'Chiefs eying AFC’s playoff bye, Seahawks a postseason spot',
 'Tips for buying technology you can feel good about giving kids',
 'Wednesday December, 21, 2022',
 '50+ Secret Santa gifts for under $20',
 'My Journey with Schizophrenia: Finding Support and Treatment',
 'Seven Chiefs selected for Pro Bowl in 2023',
 'Zelenskyy to Congress: Against all odds, Ukraine still stands',
 "The Jan 6. committee's final report details criminal referrals against Trump",
 'LA jury convicts Tory Lanez of shooting Megan Thee Stallion',
 'West Nodaway board votes to oppose open enrollment',
 'Two years after Hawley’s objection, Blunt advances changes to election certification',
 'Spasticity 101: A debilitating yet treatable common condition after a stroke',
 'Police: Missouri woman tracked stolen car, killed two men',
 'Holiday travelers see delayed Christmas Eve flights out of St. Louis',
 'Cassville Democrat',
 'Chiefs roll through Seahawks for 12th win of season',
 "5 arrested in deadly shooting at Minnesota's Mall of America",
 "Chiefs eying AFC's playoff bye, Seahawks a postseason spot",
 'At least 18 die as storm brings rain, snow, cold across US',
 "Pope on Christmas: Jesus was poor, so don't be power-hungry",
 'PUBLIC ANNOUNCEMENT',
 'FTX founder Bankman-Fried allowed $250M bond, house arrest',
 'What’s Coming & Going From Hulu in January 2023',
 'Canada condo killer faced possible eviction before shooting',
 'Zelenskyy thanks ‘every American,’ sees ‘turning point’',
 'Chiefs eyeing playoff bye, Seahawks a postseason spot',
 'The Best-Paying American Cities for Accountants',
 'Sikeston Middle School announces its honor roll for second quarter',
 'Doctor gives advice for Chiefs fans going to Saturday’s game',
 '30 best Netflix shows for binge-watching over the holidays',
 'Police report 12-20-22',
 'Police report 12-22-22',
 'Christmas Greetings 2022',
 'BenFred: Picking the perfect gifts for St. Louis sports figures, whether they want them or not',
 'Fire report 12-21-22',
 'Friday, December 23, 2022',
 'Warming centers, overnight shelters available in Columbia, Jefferson City',
 '30 dogs rescued in raid on Cass County property',
 'Out of the past: Dec. 23',
 'Jan. 6 witness recounts pressure campaign from Trump allies',
 'China sanctions 2 US citizens over action on Tibet',
 'Conservation dept. reports 16,032 deer harvested during antlerless hunting',
 'Trump’s taxes: House OKs bill on IRS audits of presidents',
 'Out of the past: Dec. 20',
 'Looking to the Future of Advanced Mobility',
 'States where Americans work the longest weeks',
 'Gas prices drop in time for Christmas',
 'Fire report 12-22-22',
 'Taliban bar women from university education in Afghanistan',
 'Community Calendar',
 "Chamber brings 'Game of Life' to 8th graders",
 'What the freezing weather means for holiday football games',
 'First Alert: Winter Storm With Triple Threat of Snow, Cold & Wind',
 'Winter arrives with snow, wind, bitter temperatures',
 "Hearing on FTX founder's extradition to US set for Wednesday",
 'Man at JFK Airport tried to hide gun in jars of peanut butter, TSA says',
 'Holiday gnomes are gaining in popularity this holiday season, here’s how you can join the craze',
 'A dream come true: Family friend helps fans from Tulsa attend Chiefs game',
 'Out of the past: Dec. 21',
 'Holiday Halitosis: Causes and Combatants of Bad Breath',
 "Missouri blows out Illinois for Braggin' Rights",
 'Royals downtown stadium design consultant talks location, timeline',
 'Chiefs gift Andy Reid a cheeseburger following win',
 "Zelenskyy thanks 'every American,' sees 'turning point'",
 "Santa's reindeer get movement approval from state",
 'Holiday lights take a special meaning deep in the Adirondack Mountains',
 'Migrants dropped near VP Harris home on frigid Christmas Eve',
 'Bidens visit patients at Children’s National Hospital',
 'Hartman throws 3 TD passes, Wake Forest beats Missouri',
 'Kremlin warning: More US arms to Ukraine will aggravate war',
 'New Missouri treasurer 1st person of color in state office',
 "Germany returns looted artifacts to Nigeria to rectify a 'dark colonial history'",
 'How much do Americans spend on entertainment?',
 'Zillow names local city most popular search in 2022',
 'Nicole Scherzinger is dreaming of a Tahiti honeymoon',
 'Police report 12-25-22',
 'Police report 12-23-22',
 'Police: St. Louis Co. woman tracked stolen car, killed two men',
 'Man pleads guilty to DWI in crash that killed MU student',
 "Water pressure 'fluctuating' in Jackson amid frigid weather",
 'Wyoming eyes fourth straight bowl win vs. Ohio in Arizona Bowl',
 'US asks court to end asylum limits, with a short delay',
 'Chiefs QB Patrick Mahomes shares story of daughter Sterling meeting Santa',
 'Man wanted, accused of shooting at trooper in southeast Mo.',
 'Jan. 6 panel urges Trump prosecution with criminal referral',
 'Wreaths Across America honors vets in Jackson',
 'New MODOT plow drivers get their first big winter storm',
 'Packers continue last-ditch playoff push at Miami',
 'Saturday December, 24, 2022',
 'Fire report 12-25-22',
 'China has stopped publishing daily COVID data amid reports of a huge spike in cases',
 'Tuesday December, 20, 2022',
 'Holiday procrastinators are back in force. Blame inflation.',
 'Ukraine’s Zelenskyy preparing to visit DC on Wednesday',
 'City of Columbia names new parks and recreation director',
 'Migrants warmed by the community as freezing temperatures linger in El Paso',
 'Florida teen beats mom with frying pan because she told him to clean his room',
 'Beyonce performing first concert in five years',
 'Tesla offers rare year-end discounts on 2 top-selling models',
 'If you’re a procrastinator, you’ll love these great last-minute gift deals on sale today',
 'Giants promote former All-Pro Landon Collins to active roster',
 'Out of the past: Dec. 24',
 "Argentina's government defends chaotic World Cup parade",
 'Jan. 6 report blames Trump, aims to prevent return to power',
 'Facebook parent Meta will pay $725M to settle user data case',
 'John Cleese turned down chance to write ABBA musical',
 "In Florida, 'health freedom' activists exert influence over a major hospital",
 'Missouri utility company asks customers to conserve',
 'Speak Out 12-21-22',
 'Biden heads to Mexico next month for leaders’ summit',
 'Missouri, Wake Forest eye opportunity in Gasparilla Bowl',
 "'A very hard road ahead' for China as COVID-19 cases spiral",
 'At least 1.5 million people without power in U.S. due to winter storm',
 'Boone County government offices will be closed Friday, Courthouse will be open',
 'Migrants in Massachusetts bring together Venezuelans living in the U.S.',
 "Seahawks' Kenneth Walker III questionable, Tyler Lockett out",
 'Harvey Weinstein found guilty of rape in Los Angeles trial',
 'Vane to lead Kearney congregation as bishop',
 'Saturday, December 24, 2022',
 'Signing period roundup: QB Arch Manning to Texas now official',
 'Public Record',
 '5 regalos para entusiasmar y cautivar a sus pequeños artistas en esta temporada festiva',
 'Spices fill holidays with aroma, memories',
 'Fitness expert shares top safety tips for outside winter exercising',
 'Missouri governor taps political newcomer to serve as next state treasurer',
 'Congress passes safety regulations on duck boats, spurred by 2018 Branson tragedy',
 'Community Christmas Meal Postponed Until Monday',
 'U.S. to Release Flu Meds From National Stockpile to Ease Shortages',
 'December 21, 2022 edition',
 'Reports: Sean Payton assembling staff for planned NFL return',
 '10 extremely last-minute Christmas gifts that can still ship in time',
 'Glass act: Scientists reveal secrets of frog transparency',
 'Around Town Chillicothe – Chillicothe News Constitution-Tribune',
 "EXPLAINER: What's the debate over releasing Trump's taxes?",
 'Robert E. Hagen',
 'Police report 12-21-22',
 'Stepping Out Of The Shadows Of HIV',
 'Is Meal Kit Delivery Still Worth the Cost?',
 "Bidens visit patients at Children's National Hospital",
 'A sense of self: Burna Boy brings Nigerian sounds to a global audience',
 '“Avatar: The Way of Water”',
 'Keeping the Holidays Cozy and Bright',
 'Operation Underground Railroad (O.U.R.) Founder Tim Ballard Reveals Details of Child Trafficking Raids in Ecuador, Mexico, Ukraine',
 'December 24, 2022 edition',
 "Ukraine's Zelenskyy preparing to visit DC on Wednesday",
 '‘Top Gun,’ ‘Black Panther’ advance in Oscars shortlist',
 'U.S. Postal Service to invest $9.6 billion in electric vehicle fleet',
 'Two dead, 11 hurt after magnitude-6.4 California earthquake',
 'Which smart home devices are actually worth it?',
 'EXPLAINER: Arctic blast sweeps US, causes bomb cyclone',
 'EXPLAINER: Arctic blast sweeps US, bomb cyclone possible',
 'Spire asks customers to conserve energy due to dramatically lower temperatures',
 'Massive winter storm brings rolling blackouts, power outages',
 'Hartman throws 3 TD passes, Wake Forest beats Missouri 27-17',
 'Kansas City charter school fires teacher accused of using racial slur',
 'Early morning apartment fire at Russellville Senior Housing',
 'Fire report 12-23-22',
 'Oregon LB Noah Sewell declares for NFL draft',
 'TRAFFIC ALERT: I-44 in Crawford County, Mo., reopens following crash involving several semis',
 'St. Louis County police officer killed in apparent murder-suicide',
 'Ryan Tannehill out, Titans to start QB Malik Willis vs Houston',
 'Speak Out 12-22-22',
 'Chris Pratt stung on eyeball by a bee',
 'Wednesday, December 21, 2022',
 'Investigator in notorious Missouri murder case faces charges',
 'Hall of Fame RB Franco Harris dies at 72',
 'Destination Kohler: 7 ways to enjoy a Midwest winter wonderland',
 '‘A very hard road ahead’ for China as COVID-19 cases spiral',
 'Tuesday, December 20, 2022',
 'With recreational weed legal, officials expect medical marijuana licenses to dwindle',
 'Drought, inflation won’t stop farm income from hitting record highs, USDA says',
 'Media Views: Joe Buck says no to calling Cardinals games on BSM; Bob Costas not interested',
 'Why I prioritize flu vaccination for my family: A mother’s tragic story',
 'Gabe Huffington named new director of Parks and Recreation',
 "Israel's Netanyahu says he has formed new government",
 'Out of the past: Dec. 25',
 'Need a last minute gift idea? 5 reasons to try a thrift store | PennyWise podcast',
 '8 consejos para evitar congelación de tuberías y presas de hielo este invierno',
 '100+ best gifts for kids to shop this holiday season',
 'Best Lego DC Comics Super Heroes Sets',
 'Zelenskyy to Congress: Against all odds Ukraine still stands',
 'DNR awards $50,000 grant to Sturgeon for city to evaluate wastewater system improvements',
 'Roy Blunt assesses the promise and peril of finding legislative consensus',
 'Hearing on FTX founder’s extradition to US set for Wednesday',
 'Townsend earns AFC Special Teams Player of the Week',
 'Residents frustrated by increasing used vehicle tax bills',
 'Out of the past: Dec. 22',
 'UK warns people to ‘not get so drunk’ on the holidays because of ambulance strike',
 'Tory Lanez convicted in Megan Thee Stallion’s shooting',
 'Calendar Contest',
 'Stay safe and healthy in extreme cold',
 'JCAC graduates 16 in bell-ringing ceremony',
 'Hunters kill three elk during archery, firearms season in Missouri',
 'Gates excited for first taste of Missouri-Illinois rivalry',
 'True North expanding to meet post-pandemic needs',
 'Four more people file for CPS board candidacy',
 'These fast food places, restaurants will be open on Christmas Day 2022',
 'Porn site founder accused of sex trafficking caught in Spain',
 'Caring for Older Adults: Managing heart attack treatment and follow-up care',
 '12 Questions To Ask Before Powering Up The Snow Thrower',
 'Storm adds uncertainty to strong holiday travel demand',
 'Biden heads to Mexico next month for leaders summit',
 '50+ white elephant gifts for under $20 for your office holiday party',
 'Best Elf on the Shelf sets',
 '‘Avatar: The Way of Water’',
 'Fire damages three apartments in Jefferson City',
 'Pope on Christmas: Jesus was poor, so don’t be power-hungry',
 'The best wireless headphones deals today',
 'US: Taliban release 2 Americans in ‘goodwill gesture’',
 'Cardinals QB Trace McSorley to start vs. Buccaneers',
 'Murder suspect arrested, charged with drug distribution while out on bond',
 'Over 20 vehicles involved in crash on I-44 near Lebanon',
 'Israel’s Netanyahu says he has formed new government',
 'Missouri Football Coaches Association all-state teams',
 'No injuries reported after structure fire in Greenview',
 'Missouri National Guard alerted to support Missourians in Winter Storm',
 'Missouri AG takes outsized role in opposing prosecutors in wrongful conviction cases',
 'Group of unhoused people in St. Louis file suit to block new homelessness law',
 'Congress approves new election rules in Jan. 6 response',
 'Japan adopts plan to maximize nuclear energy, in major shift',
 'Woman charged with punching, killing Potosi paramedic at South City Hospital',
 'EXPLAINER: What’s the debate over releasing Trump’s taxes?',
 'EXPLAINER: What happens if COVID asylum restrictions end?',
 'S&P 500 closes out dismal year with worst loss since 2008',
 'The Most Unionized Industries in the U.S.',
 'Chiefs prepare to face Broncos after firing of head coach',
 'Arkansas withstands furious Kansas rally, wins Liberty Bowl',
 'Florida Official Tapped To Lead Arkansas Education Agency',
 '2023: Year of the sparkling amaryllis',
 'Southwest cancels more flights, draws federal investigation',
 'North Macedonia takes emergency anti-pollution steps',
 'St. Louis Blues defenseman Torey Krug out for 6 weeks',
 '22 of the biggest scientific discoveries of 2022',
 'MU safety Carlies announces decision to return in 2023',
 'US to require travelers from China to show negative Covid-19 test result before flight',
 'Encore: NPR shares its favorite musical moments from 2022',
 "Adam Scott admits to battling 'rabid insecurities'",
 'Jefferson City police to have more officers on roads for New Year’s weekend',
 'Kansas City Chiefs surprise Salvation Army volunteer with Super Bowl LVII tickets',
 'Pakistan troops search for attackers after 6 soldiers killed',
 "Probe: Alzheimer's drug approval 'rife with irregularities'",
 'Out of the past: Dec. 31',
 'Airline breakdown upends holiday leave for service members',
 'Belarus hands opposition activists prison terms in absentia',
 'Iraq prime minister orders crackdown on trademark violations',
 "Southwest Airlines' #epicfail takes social media by storm",
 'Most Popular Affordable Cities for Retirees',
 'Missouri judge hands Planned Parenthood a Medicaid win',
 'What we know about the suspect arrested in connection with the Idaho college student killings',
 'Jobs projected to grow the most in the next decade',
 'Brown scores 30 as Mizzou blows out No. 19 Kentucky 89-75',
 "DONNA ERICKSON'S BEST BITES: Celebrate new year with grapefruit Baked Alaska",
 '‘Chicago Med’: Will Colin Donnell Return as Connor?',
 "Colts stand in way of Giants' first playoff berth since '16",
 'Veteran wins six lottery prizes of $25K a year for life -- in the same drawing',
 'Calendar Contest',
 'Out of the past: Dec. 30',
 'Newspaper carrier aids in Christmas Eve fire response',
 'Drawing nears for $685 million Mega Millions prize',
 'Influential Supreme Court decisions decided by 1 vote',
 'What Was the Best New Hallmark Holiday Movie of 2022? (POLL)',
 'Chiefs-Broncos: What to know if you’re going to Arrowhead on New Year’s Day',
 'Baby dies from December crash in Osage County',
 'Sonny Dykes, TCU to honor Mike Leach with pirate decals',
 'Flight canceled? Experts share some advice about what to do',
 'Chiefs expect WR Hardman back on field after nearly 2 months',
 'Brown scores 30 as Mizzou blows out No. 19 Kentucky',
 "Congressional Report Slams FDA, Drugmaker Over Approval of Alzheimer's Drug Aduhelm",
 'Respiratory viruses could surge following the holidays, public health experts warn',
 'Woman killed in Morgan County crash on Christmas Eve',
 'For 3 big Alabama newspapers, the presses are grinding to a halt',
 'Southwest Airlines flight cancellations continue to snowball',
 'Supreme Court rules pandemic-era immigration policy will remain in effect',
 "Andy Cohen claims to have inspired James Corden's set",
 'On Your Side: What to know before you join a gym',
 'KCI travelers happy to see Southwest flights back to normal',
 'Recycling your Christmas trees and holiday decor',
 'Five vying for three seats on Jefferson City School Board',
 'N Korea fires 3 missiles amid tensions over drone flights',
 'Could Soaking in a Hot Spring (or Tub) Ease High Blood Pressure?',
 'Eslinger prepares to represent Stone and Taney counties',
 'Missouri GOP renews push to limit transgender athlete participation in school sports',
 'Broncos fire rookie head coach Hackett after 4-11 start',
 'The AP Interview: Ukraine FM aims for February peace summit',
 'Report shines new light on execution secrecy in Tennessee',
 'Missouri lawmakers optimistic about upcoming legislative session',
 'Gateway Arch to require masks again starting Tuesday',
 'Drawing nears for $640 million Mega Millions prize',
 'Southwest: Normal flight operations to resume Friday',
 'Pelé remembered for transcending soccer around world',
 'Winter Is Tough on Your Skin - Fight Back',
 'Chrishell Stause’s mixed emotions over Selling Sunset',
 'Laws allow kids to be taken away from their parents if they fail to pay debts',
 'Co-leader of Whitmer kidnapping plot gets 16 years in prison',
 'Stress Can Help Bring on a Stroke, Study Shows',
 'Fire report 12-29-22',
 'Vivienne Westwood, influential fashion maverick, dies at 81',
 'Castle in the Ozarks available to buy or rent out',
 'Cole County judge blocks effort to deny Medicaid funds to Planned Parenthood',
 'Miller County roadway back open after truck overturns spilling pigs',
 'Egg prices jumped nearly 50% in 2022, data shows',
 "What's ahead for the war in Ukraine in 2023",
 "Jo Mersa Marley, Bob Marley's grandson, dies at 31",
 'Flush with gift cards and holiday cash? Check out these popular gifts you can get yourself',
 'Israel’s Netanyahu back in power with hard-line government',
 'Best movies of 2022, according to film lovers',
 'Biden signs $1.7 trillion bill funding government operations',
 'Saturday, December 31, 2022',
 'Rep. Basye seeks CPS board candidacy as district looks into legality',
 'Cassville Democrat',
 'Stocks close lower on Wall Street, adding to recent losses',
 'Biden pardons 6 convicted of murder, drug, alcohol crimes',
 "Bringing 'Fleishman Is in Trouble' to the screen",
 'Young throws for 5 TDs, Alabama tops K-State in Sugar Bowl',
 'Out of the past: Dec. 28',
 "Missouri deer hunting season regulations are changing in 2023. Here's why.",
 '10 top-rated blenders and smoothie accessories to help you kick off your New Year’s health goals',
 'Marijuana is legal in Missouri, but expungements for past crimes may take time',
 'Best gifts we tested and loved under $50',
 "House committee expected to release Trump's taxes Friday",
 'EXPLAINER: Kansas imposes TikTok ban on government devices',
 '11 states will cut individual income taxes in 2023',
 'Barack Obama shares 2022 playlist featuring SZA, Lizzo, Bad Bunny and more',
 '58 weak Rohingya land on Indonesian beach after weeks at sea',
 'Biden and Harris celebrate Kwanzaa in social media posts',
 'Royals sign RHP Lyles to 2-year deal, hope for many innings',
 'States, including Missouri, banking big bucks as Fed attempts to fight inflation',
 'Jo Mersa Marley, Bob Marley’s grandson, dies at 31',
 'All the Changes Coming to TV & Streaming in 2023',
 '2022 in Review',
 'Florida official tapped to lead Arkansas education agency',
 'Judge says Missouri Department of Corrections must pay $2M in sexual harassment case',
 'Tuesday, December 27, 2022',
 'Fire report 12-28-22',
 'Reports: Bears considering Big Ten head Kevin Warren for team president',
 'Tales of buried treasure in the Ozarks',
 'Loan forgiveness, hazing, vaccines: Missouri higher education bills to watch in 2023',
 'Missouri Department of Corrections must pay $2M in sexual harassment case, judge says',
 'Helicopter, with 4 on board, crashes in Gulf of Mexico',
 'The cost of emailing your doctor, a fentanyl vaccine and endangered wildlife | Hot off the Wire podcast',
 'New wholesale store opens its doors in Sedalia',
 'Rep.-elect George Santos faces growing anger from New York voters',
 'Graham Coxon: I didn’t deserve to have fun in Blur’s hey day',
 'Democrat wins Arizona attorney general race after recount',
 'College scandal middleman blames ‘winning at all costs’',
 'House committee expected to release Trump’s taxes Friday',
 'No. 5 Alabama, No. 9 K-State have plenty to prove in Sugar Bowl',
 'Is it time to get your home’s ducts in order?',
 'Wednesday December, 28, 2022',
 'How to improve your basement and other spaces by leveling uneven floors',
 '10 statement armchairs that will brighten any room',
 'How Barbara Walters helped Americans understand their presidents',
 '2 miles of border fencing installed in El Paso area as migrants surge around Texas amid Title 42 limbo',
 "Vatican says health of retired pope Benedict XVI 'worsening'",
 'Cards turn to QB David Blough to start vs. Falcons',
 'A look back at 2022 in Mid-Missouri',
 'Putin claims Moscow ready for Ukraine talks as attacks go on',
 'Best playpen',
 'Wake Forest QB Sam Hartman in portal; Notre Dame next stop?',
 'Central Taney County Fire Protection District seeks new chief',
 'Hartman retired from Guard as sergeant major',
 'New rebalance and dizziness center in Sikeston is one of only three in the state',
 '2023 public domain debuts include last Sherlock Holmes work',
 'Out of the past: Dec. 26',
 'Supreme Court keeps immigration limits in place indefinitely',
 'December 28, 2022 edition',
 'Wednesday, December 28, 2022',
 'Christmas tree recycling is a good alternative to landfills',
 'Biden heading to US Virgin Islands to relax between holidays',
 'Vatican says health of retired pope Benedict XVI ‘worsening’',
 'Police raid hospital room of terminally ill patient',
 'Supreme Court says Trump-era border restriction will remain in effect while legal challenges play out',
 'Best women’s winter coat for extreme cold',
 'FDA recalls blood pressure medication over cancer risks',
 'China to resume issuing passports, visas as virus curbs ease',
 'Imposter scams, robocalls and credit card fraud among top cons of 2022',
 'Miller County roadway back open after truck overturns, spilling pigs',
 'Benedict, the former pope, dies at 95',
 'Southwest flight canceled? Here’s what you can do',
 'December 31, 2022 edition',
 'Suspect in Paul Pelosi attack pleads not guilty to state charges',
 "China's foreign minister signals deeper ties with Russia",
 'Ian Tyson, half of Ian & Sylvia folk duo, dies at age 89',
 'Frustrations boil over as Southwest Airlines passengers in St. Louis see cancelations, delays, lost bags',
 'Alfalfa sprouts being recalled after salmonella outbreak',
 'Saturday December, 31, 2022',
 'Out of the past: Dec. 27',
 'Glittery eyeshadow palettes to give you the perfect New Year’s Eve look',
 'DIVAS ON A DIME: Celebrate new year with inspired cocktail creations',
 "Lack of info on China's COVID-19 surge stirs global concern",
 'Brian Laundrie’s mother offered shovel to help bury body, claims attorney for Gabby Petito’s parents',
 'Buffalo holds off Georgia Southern to win Camellia Bowl',
 'All Songs Considered counts down the top songs of 2022',
 'Americans expected to return up to $170 billion in holiday gifts',
 'Davis Street construction in Jefferson City extended through Jan. 31',
 'Helicopter with 4 on board crashes in Gulf of Mexico',
 "College scandal middleman blames 'winning at all costs'",
 '2022 in review',
 'Dolphins QB Tua Tagovailoa in concussion protocol again',
 'Bill forcing feds to fix prison cameras is signed into law',
 'Road deaths in Missouri exceed 2021 numbers',
 'Pelé dead aged 82 after losing cancer battle',
 "From Springfield to 'Babylon': Missouri State grad Troy Metcalf appears in Brad Pitt film",
 'Tuesday December, 27, 2022',
 '• COMMUNITY CALENDAR',
 'Beverly Guenther',
 "As 'The King,' Pelé enchanted fans and dazzled opponents",
 'Report: No suspension for Chargers S Derwin James',
 '25 celebrities you may not know are nonbinary',
 'Saudi Arabian club Al Nassr signs Cristiano Ronaldo',
 "Vatican: Benedict XVI lucid, stable, but condition 'serious'",
 'Cinco cosas a seguir con los St. Louis Cardinals, Blues y más allá en 2023',
 'Community Calendar',
 'New Year’s superstitions that might be worth trying',
 'Gateway Arch reinstating mask mandate on Tuesday',
 'Military police enforce driving ban in snow-stricken Buffalo',
 'Mega Millions jackpot up to $640 million after no big winner',
 'One cool cave in Missouri you’re not allowed to enter',
 'Public Record',
 '“Puss in Boots: The Last Wish”',
 '18-year-old killed after car struck by train in Phelps County',
 'Idaho college killings suspect is criminology PhD student',
 'Historians reflect on Harry Truman’s passing 50 years later',
 'Baby, it’s cold outside: Parents deliver daughter in subzero temperatures',
 "Ukraine is America's latest stalemate war",
 'Volkswagen recalls Beetles to replace Takata air bags',
 'Report: 2 missing after Austria avalanche, fewer than feared',
 'Serbia places its troops on Kosovo border on combat alert',
 'Here are the worst states for flu this year',
 "Israel's Netanyahu back in power with hard-line government",
 'Cashing in holiday gift cards? These 21 deals are worth using them on at big retailers',
 'Best hoverboards',
 'Friday December, 30, 2022',
 'These 17 sparkly outfits make the perfect last-minute New Year’s Eve look',
 'Finding more personal growth, progress in 2023',
 'Measles Outbreak in Ohio Infects 82 Kids, Most of Them Unvaccinated',
 'Long COVID: Could mono virus or fat cells be playing roles?',
 'Liberty Bowl pits Kansas vs Arkansas for 1st time since 1906',
 'December 29, 2022 edition',
 'Waitress surprised with $1K tip on Christmas day',
 'Benedict XVI, first pope to resign in 600 years, dies at 95',
 'How to keep your dog safe and warm this winter',
 'Out of the past: Dec. 29',
 'LGBTQ+ people who made history in 2022',
 'Friday, December 30, 2022',
 'Where refugees to the US come from and why',
 'NY opens its first legal recreational marijuana dispensary',
 "DONNA ERICKSON'S BEST BITES: Serve up Caesar salad for whole family",
 '‘Atmospheric river’ dumps heavy rain, snow across California',
 'Border crisis: Shootouts, pursuits, break-ins surge in rural Texas counties',
 'Chiefs: Mahomes reaches 5000 yards passing once again',
 'Elon Musk buying Twitter has been all over the news lately. Are you on Twitter?',
 'Stable Chiefs face reeling Broncos in midst of playoff push',
 "Taxes fall, wages rise and jaywalking OK'd by new state laws",
 'Time zone by time zone, another new year sweeps into view',
 'Newsletters',
 'East St. Louis man shot dead in Carr Square',
 'US to require travelers from China to show negative Covid-19 test result before flight',
 'Despite changes, Broncos stuck with same old result in KC',
 'No. 1 Georgia rallies to beat No. 4 Ohio State 42-41 in semi',
 'What they said: A look back at the interviews of 2022',
 'Falcons WR Cameron Batson facing five charges',
 'Kirstie Alley, Emmy-winning ‘Cheers’ star, dies at 71',
 "Kirstie Alley, Emmy-winning 'Cheers' star, dies at 71",
 'Is Meal Kit Delivery Still Worth the Cost?',
 'Coronavirus News',
 'Chief justice: Judges’ safety ‘essential’ to court system',
 'Local News',
 'Hartman scores twice to help Wild beat Blues 5-2',
 'Calendar Contest',
 'Watch: Lone Star NYE fireworks show from Dallas',
 'News',
 'Ozark Adventures',
 'Community Calendar',
 'Morning Show',
 'Mardi Gras-themed party kicks off new year',
 "Make Curbing Allergies, Asthma Your New Year's Resolution",
 'Plaintiff attorneys: Lawsuits against DOD to continue despite recision of vaccine mandate',
 'Wednesday, December 14, 2022',
 'Chiefs: Mahomes reaches 5,000 yards passing once again',
 "Columbia men's basketball drops road game to No. 21 Kansas Wesleyan",
 'How To Choose the Best Weight Loss Plan for Your 2023 Goals',
 'Benedict XVI, first pope to resign in 600 years, dies at 95',
 'Friday, July 2, 2021',
 'Area Scoreboard',
 'Police attacked by man with machete near Times Square on New Year’s Eve',
 "New laws take effect for Columbia residents New Year's Day",
 "How to actually keep your New Year's resolutions",
 "Brazil's Lula sworn in, vows accountability and rebuilding",
 'Buffalo Reflex',
 'Foo Fighters confirms band will be ‘different’ after Taylor Hawkins death, but vows to see fans ‘soon’',
 'BHS Graduation 2022',
 '14 killed in attack on Mexican border prison',
 'Highlights from the life of Pope Emeritus Benedict XVI',
 'Democrat Kathy Hochul sworn in as elected New York governor',
 'Christine McVie, Fleetwood Mac singer-songwriter, dies at 79',
 'Evacuation warnings amid flooding after California storm',
 'BHS Graduation 2021',
 'Wednesday, December 21, 2022',
 'Mega Millions jackpot climbs to $785M after no big winner',
 "Madi Diaz talks 'Same History, New Feelings'",
 "Three NYPD officers injured in machete attack near New Year's Eve celebration in Times Square, officials say",
 'Wednesday December, 28, 2022',
 'Taxes fall, wages rise and jaywalking OK’d by new state laws',
 'Santos should consider resigning, veteran GOP lawmaker says',
 'Chiefs’ Jerick McKinnon sets milestone with TD reception',
 '2 dead and 4 others injured in New Year’s Day shooting in Florida',
 'No. 3 TCU upsets No. 2 Michigan 51-45 in wild CFP semifinal',
 "'Atmospheric river' dumps heavy rain, snow across California",
 'Andrew Bailey to be sworn in as new Missouri AG',
 'Citizen Front Page — The Platte County Citizen',
 'State to pay prison nurse $2 million in harassment case',
 "2 dead and 4 others injured in New Year's Day shooting in Florida",
 'Mahomes surpasses 5,000 yards passing for second time in his career',
 "Live Updates | Reactions to Pope Benedict XVI's death",
 'Wednesday December, 7, 2022',
 'Singer Anita Pointer of The Pointer Sisters dies at age 74',
 "Chief justice: Judges' safety 'essential' to court system"]

In [4]:
len(exact_duplicates)

1323

In [24]:
#Finding the columns which are exact duplicates in terms of the headline, text and hostname
exact_texts = data[['title','news','hostname']]
exact_texts = exact_texts.value_counts().to_frame('counts').reset_index()
exact_texts.to_csv('exact_texts.csv')

In [26]:
exact_texts.head(30)

,title,news,hostname,counts
0,Springfield News-Leader,Your subscription supports:\n\nInvestigative r...,www.news-leader.com,31
1,"AP-AR--Arkansas Digest, 1pm update",Heres a look at how APs general news coverage ...,www.westplainsdailyquill.net,22
2,St. Louis Riverfront Times,404! Website Error\n\nWe're sorry; the page yo...,www.riverfronttimes.com,19
3,I SPY — Polk County,How well do you know Polk County?,bolivarmonews.com,5
4,PUBLIC ANNOUNCEMENT,The Polk County Courthouse will be closed Frid...,bolivarmonews.com,4
5,Public Record,CIVIL ACTIONS,bolivarmonews.com,4
6,PDF: Livestock Market Report,Launch Student Ministries is the youth group a...,houstonherald.com,4
7,The Kansas City Star,Subscribe to continue reading\n\nSave over 90%...,www.kansascity.com,4
8,PDF: This week’s public notices,Launch Student Ministries is the youth group a...,houstonherald.com,4
9,Mark Twain Tree Removal,Mark Twain Lake staff will be removing dead/ha...,www.lakegazette.net,4


In [31]:
#Removing duplicates that are the same in terms of the headline, text and hostname, but keeping the first instance
data_without_dupes = data.drop_duplicates(subset=['title','news','hostname'], keep='first')
data_without_dupes.shape

(32700, 13)

In [46]:
#Now need to catch the instances in which the scraper picked up the same story from the same publication twice (
#either because there were two different URLs used to place the story in different sections on the website 
#or for some other reason. This is a bit tricky though because some of these titles with multiple counts are actually
#series that are just named the same thing every week. Therefore, I will only look for the examples that are repeated
#twice -- these are likely the ones to be the same story that was just listed as two different URLs)

data_further_cleaning = data_without_dupes[['title','hostname']]
data_further_cleaning = data_further_cleaning.value_counts().to_frame('counts').reset_index()
data_further_cleaning.head(40)

list = []
for index, row in data_further_cleaning.iterrows():
    if row['counts']==2:
        list.append(row['title'])
list


['School board remains firm on performing arts center March opening',
 'UK warns people to ‘not get so drunk’ on the holidays because of ambulance strike',
 'Man attempting U-turn fatally shoots himself in groin',
 'Hal Vance Ross, Jr.',
 'Jackson County restaurant inspections — Blue Springs – The Examiner',
 'Man arrested in fatal shooting of Migos rapper Takeoff',
 'Goodman looking to solar to save money',
 'Show Me KC Schools’ SchoolAppKC launches for the fifth year',
 'School construction update: All’s well, so far',
 'Gas Prices Continue To Drop',
 'John Pace',
 'Arts Rolla presents the work of William Oberholtzer during the month of January',
 'Marshfield Mail',
 'City Hall to close for holidays',
 'Warren Lynn Goode',
 'Barack Obama shares 2022 playlist featuring SZA, Lizzo, Bad Bunny and more',
 'PSU, MSSU open indoor track and field season at Crimson & Gold Invitational',
 'Mexico woman injured in Audrain County accident',
 'Family catches Ring video of moose shedding its antl

In [58]:
#Removing the articles from the list from the working dataframe
data_cleaned = data_without_dupes[~data_without_dupes['title'].isin(list)]
data_cleaned.shape

#Pulling out the articles from the list and keeping only the first occurence
data_test = data_without_dupes[data_without_dupes['title'].isin(list)]
data_test = data_test.drop_duplicates(subset=['title','hostname'], keep='first')
data_test.shape

#Inserting data back into the working dataframe
data_cleaned = data_cleaned.append(data_test)
data_cleaned.shape

#Export
data_cleaned.to_csv('december_data_cleaned')

In [57]:
test2 = data_cleaned[['title','hostname']]
test2 = test2.value_counts().to_frame('counts').reset_index()
test2.head(60)

,title,hostname,counts
0,"Southeast Missourian newspaper, Cape Girardeau...",www.semissourian.com,101
1,Marshall Democrat-News,www.marshallnews.com,27
2,Cassville Democrat,www.cassville-democrat.com,19
3,South Missourian News,www.southmissouriannews.com,16
4,"The Reporter newspaper, Camdenton Missouri",www.theccreporter.com,13
5,Boone County Journal,bocojo.com,11
6,AP News in Brief at 6:04 p.m. EST,www.joplinglobe.com,11
7,What’s Up Next area sports calendar – The Exam...,www.examiner.net,7
8,Benton County Sheriff's Report,www.bentoncountyenterprise.com,7
9,Maries County Advocate,mariescountyadvocate.com,6


## Publications publishing the same articles

In [100]:
#The duplicates from the same publication should be eliminated. Now we can look at duplicate articles that appear
#across publications. These will stay in the data but it would be good to get an overall sense of them

republished_content = data_cleaned[['title']]
republished_content = republished_content.value_counts().to_frame('counts').reset_index()
republished_content.head(10)

#Need to skip some of the top results that are reflecting pieces with the same title that actually have different content
#Start at row 6

republished_articles = republished_content[6:].loc[republished_content['counts'] > 1]
republished_articles = republished_articles['counts']
republished_articles.sum()

#7,009 articles were not unique to one publication (i.e. at least one other publication shared the same article with 
#the EXACT same headline)


7009

## Close/Fuzzy Duplicates

In [101]:


fuzzy_matches = ["5 tips to consider if you're giving gift cards this holiday season | PennyWise podcast",
 "Affidavit: Oklahoma man said he killed 4 men, 'cut them up'",
 'Republican Senator files bill to address state’s high maternal mortality • Missouri Independent',
 'Affidavit: Oklahoma man said he killed 4 men, ‘cut them up’',
 'Linn Creek Missouri woman charged with abuse after child burned in stove',
 'New baby in tow, Chiefs’ Mahomes turns attention to Bengals',
 'Affidavit: Oklahoma man said he killed 4 men, ‘cut them up’ – Newstalk KZRG',
 'School shootings spur legislation for stricter gun safety regulations',
 "5 tips to consider if you're giving gift cards this holiday season",
 "Missouri Supreme Court denies stay for Kevin Johnson's execution • Missouri Independent",
 'Linn Creek woman charged with abuse after child burned in stove',
 'U.S. House votes to avert calamitous rail strike, but Senate prospects murky',
 'Republican Senator files bill to address state’s high maternal mortality',
 'House votes to avert rail strike, impose deal on unions – The Examiner',
 'Santa Claus riding the rails into Poplar Bluff',
 '2 charged in death of Missouri child who ingested fentanyl – Newstalk KZRG',
 'U.S. House votes to avert calamitous rail strike, but Senate prospects murky • Missouri Independent',
 'Kansas City’s historic northeast pushes back against proposed school closures • Missouri Independent',
 'Ashcroft criticizes Blunt support for Senate bill protecting same-sex marriage • Missouri Independent',
 'Kansas City’s historic northeast pushes back against proposed school closures',
 'Missouri Supreme Court denies stay for Kevin Johnson’s execution',
 '2 charged in death of Missouri child who ingested fentanyl',
 'House votes to avert rail strike, impose deal on unions',
 'Santa Claus riding the rails Saturdays into Poplar Bluff',
 'Ashcroft criticizes Blunt support for Senate bill protecting same-sex marriage',
 'New baby in tow, Chiefs’ Mahomes turns attention to Bengals – The Examiner',
 "Teens' brains aged faster during the first year of the pandemic, study says, and stress may be to blame",
 'School shootings spur legislation for stricter gun safety regulations • Missouri Independent',
 "Teens' brains aged faster during the first year of the pandemic, study says. Stress may be to blame",
 "'Wakanda Forever' is No. 1 for 4th straight weekend",
 'States test an electrifying idea: Roads that can recharge your EV',
 "Elizabeth Holmes' former partner gets nearly 13 years in Theranos case",
 'Trump rebuked for call to suspend Constitution over 2020 election lies',
 'Trump rebuked for call to suspend Constitution over election',
 "Experts worry expungements under Missouri's marijuana law could be procedural ‘nightmare’",
 '5 tips for donating your time or money this holiday season',
 'Missouri violated collective bargaining rights of prison guards, appeals court rules',
 'US police rarely deploy deadly robots to confront suspects',
 'Economic slowdown, tax cuts likely to limit Missouri revenue growth, officials say • Missouri Independent',
 'Summer food benefits for low-income Missouri children likely delayed until well into winter • Missouri Independent',
 'Missouri violated collective bargaining rights of prison guards, appeals court rules • Missouri Independent',
 'Chiefs could have Thuney, Toney back from injury against Broncos',
 'Prosecutor seeking to free inmate accused of hiding evidence – Newstalk KZRG',
 "‘Y'all,’ that most Southern of Southernisms, is going mainstream – and it’s about time • Missouri Independent",
 "'Wakanda Forever' No. 1 for 4th straight weekend",
 'Anti-vaxxers seek to intervene in Missouri AG Eric Schmitt’s social media lawsuit • Missouri Independent',
 'Anti-vaxxers seek to intervene in Missouri AG Eric Schmitt’s social media lawsuit',
 'Experts worry expungements under Missouri marijuana law could be procedural ‘nightmare’ • Missouri Independent',
 'Oklahoma panel votes to reject clemency for death row inmate – Newstalk KZRG',
 'Experts worry expungements under Missouri marijuana law could be procedural ‘nightmare’',
 "Confusion over Iran's religious police as women drop hijab",
 'Supreme Court justices spar in latest clash of religion and gay rights',
 'The spies who deserve to be out in the cold',
 'Rich Lowry: The spies who deserve to be out in the cold',
 'Attempt to help states ease banking for marijuana businesses stumbles in Congress • Missouri Independent',
 'Flags are to fly at half-staff Wednesday, Thursday',
 'Prosecutor seeking to free inmate accused of hiding evidence',
 'Economic slowdown, tax cuts likely to limit Missouri revenue growth, officials say',
 '18-year-old college student elected mayor of Arkansas city – Newstalk KZRG',
 'Missouri Senate President says this session will almost be a clean slate',
 'ACLU sues Missouri school district over book removal policy',
 'End of public health emergency could cause spike in uninsured Missouri children • Missouri Independent',
 'Attempt to help states ease banking for marijuana businesses stumbles in Congress',
 'US police rarely deploy deadly robots to confront suspects – The Examiner',
 'ACLU sues Missouri school district over book removal policy – Newstalk KZRG',
 '18-year-old college student elected mayor of Arkansas city',
 'Marriage equality bill heads to Biden’s desk following bipartisan U.S. House vote • Missouri Independent',
 '5 tips for donating your time or money this holiday season | PennyWise podcast',
 "Confusion over Iran's religious police is growing as women drop hijab",
 'Oklahoma panel votes to reject clemency for death row inmate',
 'End of public health emergency could cause spike in uninsured Missouri children',
 'Gary Pinkel’s Hall of Fame coaching career reversed decades of losing at Mizzou – The Examiner',
 'Jackson County to pay $5.3 million for age discrimination',
 '‘Y’all,’ that most Southern of Southernisms, is going mainstream – and it’s about time',
 'Jackson County to pay $5.3 million for age discrimination lawsuit',
 'Marriage equality bill heads to Biden’s desk following bipartisan U.S. House vote',
 'States test an electrifying idea: Roads that can recharge your EV • Missouri Independent',
 'Summer food benefits for low-income Missouri children likely delayed until well into winter',
 'Confusion over Iran’s religious police as women drop hijab',
 'NOTICE OF FILING FINAL SETTLEMENT AND PETITION FOR DISTRIBUTION',
 'NOTICE OF FILING OF FINAL  SETTLEMENT AND PETITION FOR DISTRIBUTION',
 'Missouri Senate President says this session will almost be a clean slate – Newstalk KZRG',
 'Chiefs could have Thuney, Toney back from injury against Broncos – The Examiner',
 'Justices spar in latest clash of religion and gay rights',
 'Flags are to fly at half-staff Wednesday and Thursday',
 "Gary Pinkel's Hall of Fame coaching career reversed decades of losing at Mizzou",
 'Jackson County to pay $5.3 million for age discrimination – Newstalk KZRG',
 "Holmes' former partner gets nearly 13 years in Theranos case",
 'Keystone Pipeline has history of spills, warnings and fines. Kansas spill is largest yet',
 '‘Historic moment’: Lamar Johnson wrongful conviction case goes before a St. Louis judge',
 'Latest Iowa bird flu cases push December total near 700,000',
 "Lamar Johnson's wrongful conviction case goes before a St. Louis judge: ‘Historic moment’",
 'Nearly 300 Afghan refugees from Albania will arrive in St. Louis in a few months',
 'US almost sent Reyna home from World Cup for lack of hustle',
 'Latest Iowa bird flu cases push December total near 700,000 – Newstalk KZRG',
 "Rowe gets filthy on 'Dirty Jobs' to reconnect Americans",
 'Historic moon mission ends with splashdown of Orion capsule',
 "Mike Rowe gets filthy on 'Dirty Jobs' to reconnect Americans",
 'Longtime Agape director testifies in Missouri court to stay off child abuse registry',
 'Guilty plea on hate, arson charges in Islamic center fire – Newstalk KZRG',
 'The AP Interview: Hutchinson says Trump worst choice for GOP – Newstalk KZRG',
 'Independence School District is the largest Missouri district to switch to a 4-day week',
 'VIDEO: Historic moon mission ends with splashdown of Orion capsule',
 "'Historic moment': Lamar Johnson wrongful conviction case goes before a St. Louis judge • Missouri Independent",
 'First-half defense leads Joplin Eagles to victory',
 'Federal data: Kansas oil spill biggest in Keystone history – Newstalk KZRG',
 'Longtime Agape director testifies in Missouri court to stay off child abuse registry • Missouri Independent',
 'Missouri has roughly $150 million left to distribute in federal housing assistance',
 'Chiefs prepare for top-flight Broncos defense after Bengals loss',
 'Prosecutor seeking to free inmate accused of hiding evidence',
 'Kansas City Chiefs prepare for top-flight Broncos defense after Bengals loss',
 'Scientists announce a fusion breakthrough with big implications for clean energy • Missouri Independent',
 'Local News: Bipartisan group of lawmakers push to restrict foreign ownership of Missouri farmland (12/13/22)',
 'Bipartisan group of lawmakers push to restrict foreign ownership of Missouri farmland',
 'U.S. almost sent Reyna home from World Cup for lack of hustle',
 'Scientists announce a fusion breakthrough with big implications for clean energy',
 'The AP Interview: Hutchinson says Trump worst choice for GOP',
 'Lawsuit seeks to toss Missouri law used by AG to sue schools over COVID policies',
 'Lawsuit seeks to toss Missouri law used by AG to sue schools over COVID policies • Missouri Independent',
 'Independence School District becomes the largest Missouri district to switch to a 4-day week',
 'Federal data: Kansas oil spill biggest in Keystone history',
 'Bipartisan group of lawmakers push to restrict foreign ownership of Missouri farmland • Missouri Independent',
 'St. Louis prosecutor seeking to free inmate accused of hiding evidence',
 'Missouri Republicans file bills to automate expungements for non-violent offenses',
 'Missouri has roughly $150 million left to distribute in federal housing assistance • Missouri Independent',
 'Guilty plea on hate, arson charges in Islamic center fire',
 'Keystone Pipeline has history of spills, warnings and fines. Kansas spill is largest yet • Missouri Independent',
 'Nearly 300 Afghan refugees will arrive in Missouri from Albania in a few months',
 'First-half defense leads Eagles to victory',
 'Missouri Republicans file bills to automate expungements for non-violent offenses • Missouri Independent',
 "McCarthy's race for speaker risks upending House on Day One",
 'Missouri spam calls/texts have increased, so can they be stopped?',
 'Missouri awards over $40 million in low-income housing tax credits',
 'Lamar Johnson takes the stand to argue his innocence of 1994 murder',
 'Missouri chamber encourages child care providers to offer health insurance to workers',
 'Missouri awards over $40 million in low-income housing tax credits • Missouri Independent',
 'Chiefs can clinch AFC West title with win over Texans – Newstalk KZRG',
 'Man killed after crash in Macon County',
 'Voters embraced affordable housing initiatives. Advocates say Congress should do the same • Missouri Independent',
 'Report: Chiefs’ Patrick Mahomes to invest in Major League Pickleball franchise',
 'Missouri Republican pushes for constitutional change to recognize same-sex marriages',
 'Spam calls/texts have increased, so can they be stopped?',
 "Kevin McCarthy's race for speaker risks upending House on Day One",
 'Man killed after crash on Highway 5 in Macon County',
 'Boy found buried under Arkansas house; mother, man arrested',
 'Congress OKs bill forcing feds to fix broken prison cameras',
 'Chiefs can clinch AFC West title with win over Texans – The Examiner',
 'Veteran Kansas City officer chosen to lead police department',
 'Company reopens most of pipeline following Kansas oil spill',
 "Photos: Messi's hometown in Argentina yearns for World Cup victory",
 'Sports wagering, video lottery games again in conflict as legislature prepares for 2023 session • Missouri Independent',
 'US buying 3M barrels of oil to start replenishing reserves',
 'Missouri chamber encourages child care providers to offer workers health insurance',
 'Ellen DeGeneres "heartbroken" over the death of Stephen "tWitch" Boss',
 'Veteran Kansas City officer chosen to lead police department – Newstalk KZRG',
 'Port district posed as benefit to MSP project',
 'Homicide detective in Lamar Johnson’s case says he had no evidence',
 'Boy found buried under Arkansas house; mother, man arrested – Newstalk KZRG',
 "Messi's hometown in Argentina yearns for World Cup victory",
 'Voters embraced affordable housing initiatives. Advocates say Congress should do the same',
 'McCarthy’s race for speaker risks upending House on Day One',
 'Homicide detective in Lamar Johnson’s case says he had no evidence • Missouri Independent',
 'Chiefs Patrick Mahomes to invest in Major League Pickleball franchise',
 'Kyrou scores 2, Blues beat Flames 5-2 for third straight win',
 'Congress OKs bill forcing feds to fix prison cameras',
 "Ellen DeGeneres 'heartbroken' over death of Stephen 'tWitch' Boss",
 'Missouri Republican pushes for constitutional change to recognize same-sex marriages • Missouri Independent',
 'Sports wagering, video lottery games again in conflict as legislature prepares for 2023 session',
 'Lamar Johnson takes the stand to argue his innocence of 1994 murder • Missouri Independent',
 'Heartland Port district posed as benefit to MSP project',
 'Chiefs can clinch AFC West title with win over Texans',
 'US buying 3 million barrels of oil to start replenishing reserves',
 'Kyrou scores twice, Blues beat Flames 5-2 for third straight win',
 'Company reopens most of pipeline following Kansas oil spill – Newstalk KZRG',
 'Missouri governor taps political newcomer to serve as next state Treasurer',
 'Police found missing kidnapped baby when they stopped to eat',
 'Federal spending bill ensures one year continuous coverage for Missouri kids in Medicaid, CHIP',
 'Pointing to ‘flaw’ in Missouri death penalty, judges ask to stop execution',
 'Senate confirms new U.S. ambassador to Russia',
 'Environmental enforcement has fallen off under Biden, report says',
 'Police investigate artillery shell found at Branson, Mo. wilderness park',
 'Regulators back streamlined system to challenge rail rates',
 'Netflix to put a stop to password sharing',
 'Botched executions reached ‘astonishing’ high in 2022, report finds • Missouri Independent',
 'Botched executions reached ‘astonishing’ high in 2022, report finds',
 'St. Louis alderman describes encounter with woman he says tried to rob him',
 'Arkansas, Missouri gas prices continue to drop for holiday week',
 'Winter storm approaching, so now is the time to prepare',
 'Judge grants injunction keeping longtime Agape director off Missouri child abuse registry',
 'Police investigate artillery shell found at Branson, Mo. wilderness park; 1 arrested',
 'Drone no-fly zone issued over Keystone oil spill site in Kansas',
 'Senate confirms new US ambassador to Russia',
 'FEMA: Winter Storm Approaching So Now Is The Time To Prepare',
 'Missouri governor taps political newcomer to serve as next state treasurer',
 'Drone no-fly zone issued over Keystone oil spill site in Kansas • Missouri Independent',
 'Kansas City Chiefs QB Patrick Mahomes shares story of daughter Sterling meeting Santa',
 'Pointing to ‘flaw’ in Missouri death penalty, judges ask to stop woman’s execution',
 'US economy grew 3.2% in third quarter, an upgrade from earlier estimate',
 'Federal spending bill ensures one year continuous coverage for Missouri kids in Medicaid, CHIP • Missouri Independent',
 'State Fire Marshall Cautions Safety As Bitter Cold Grips Missouri This Week – Newstalk KZRG',
 'Local News: Stoddard County officials promise to improve property tax withholdings, agreements with non profits after audit findings (12/23/22)',
 'Southern Boone school board selects Tim Roth as new superintendent',
 'Alderman describes encounter with woman he says tried to rob him',
 'Missouri governor taps political newcomer to serve as next state treasurer • Missouri Independent',
 'Oath Keepers founder testifies his group was acting in ‘counter-revolution’ • Missouri Independent',
 'Oath Keepers founder testifies his group was acting in ‘counter-revolution’',
 'Chiefs QB Patrick Mahomes shares story of daughter Sterling meeting Santa',
 "Holiday travel upended as weather forecasters warn of 'bomb cyclone'",
 'Gov. Parson activates Missouri National Guard in response to winter weather storm',
 'Gov. Parson activates Missouri National Guard in response to statewide winter weather storm',
 'Stoddard County officials promise to improve property tax withholdings, agreements with non profits after audit findings',
 'Southern Boone school board selects Dr. Tim Roth as new superintendent',
 'US economy grew 3.2% in Q3, an upgrade from earlier estimate',
 'Regulators Back Streamlined Challenge To Rail Rates',
 'Missouri gas prices continue to drop for holiday week',
 'Netflix to put a stop to password sharing in 2023',
 'Huge $1.7 trillion spending package passes in U.S. Senate, backed by both parties',
 'State Fire Marshall Cautions Safety As Bitter Cold Grips Missouri This Week',
 "Holiday travel upended as forecasters warn of 'bomb cyclone'",
 'Judge grants injunction keeping longtime Agape director off Missouri child abuse registry • Missouri Independent',
 'Police found kidnapped baby when they stopped to eat',
 'Huge $1.7 trillion spending package passes in U.S. Senate, backed by both parties • Missouri Independent',
 'Holiday travel upended as forecasters warn of ‘bomb cyclone’',
 'Environmental enforcement has fallen off under Biden, report says • Missouri Independent',
 '‘Emotional roller coaster’: Delays in effort to shut down Agape dishearten former students',
 '‘Emotional roller coaster’: Delays in effort to shut down Agape dishearten former students • Missouri Independent',
 'During the holiday season, make time to assess, adjust and commit',
 'Kathy Whitworth, winningest golfer in history, dies at 83',
 'Hawley on House Democrats releasing Trump’s personal tax info – Newstalk KZRG',
 'Cole County judge blocks effort to deny Medicaid funds to Planned Parenthood',
 'States banking big bucks as Fed attempts to fight inflation – Newstalk KZRG',
 'Judge blocks effort to deny Medicaid funds to Planned Parenthood',
 'New MO Law to Make Sleeping on State Land a Crime',
 'Alan Pratzel urged education and diversion as Missouri’s chief disciplinary counsel',
 'During the holiday season, make time to assess, adjust and commit • Missouri Independent',
 'Maryland Rep. Jamie Raskin says he’s been diagnosed with lymphoma',
 'Chiefs expect WR Hardman back on field after nearly two months',
 'Benedict XVI, first pope to resign in 600 years, dies',
 'Stay proactive about your health with multi-cancer early detection screening',
 'Hansen Enmanuel, college player with 1 arm, becomes an inspiration',
 'Massive Southwest Airlines disruption leaves customers stranded and call centers swamped',
 'Maryland Rep. Raskin says he’s been diagnosed with lymphoma',
 'St. Louis man pleads guilty to heroin trafficking in Springfield',
 'Massive Southwest Airlines disruption leaves customers stranded, call centers swamped',
 'Missouri banking big bucks as Fed attempts to fight inflation',
 'States banking big bucks as Fed attempts to fight inflation',
 'St. Louis man pleads guilty to heroin trafficking in Springfield – Newstalk KZRG',
 'Kathy Whitworth, the winningest golfer in history, dies at 83',
 'Stable Chiefs face reeling Broncos in midst of playoff push',
 'Alan Pratzel urged education and diversion as Missouri’s chief disciplinary counsel • Missouri Independent',
 'Chiefs expect WR Hardman back on field after nearly 2 months',
 'Missouri Lottery player wins $4.2M Lotto jackpot using handpicked numbers',
 'Kansas governor bans TikTok from state-owned devices',
 'Royals sign RHP Lyles to 2-year deal, hope for many innings',
 'Stable Chiefs face reeling Broncos in midst of playoff push – The Examiner',
 'Hawley on House Democrats Releasing Trump’s Personal Tax Info',
 'How to stay proactive about your health with multi-cancer early detection screening',
 'Benedict XVI, first pope to resign in 600 years, dies at 95',
 'Enmanuel, college player with 1 arm, becomes an inspiration',
 'States, including Missouri, banking big bucks as Fed attempts to fight inflation',
 'Kansas governor bans TikTok from state-owned devices – Newstalk KZRG',
 'Royals sign RHP Lyles to two-year deal, hope for many innings',
 'Missouri Lottery player wins $4.2 million Lotto jackpot using handpicked numbers',
 'New MO law to make sleeping on state land a crime – Newstalk KZRG',
 "Maryland Rep. Raskin says he's been diagnosed with lymphoma",
 'Capitol Perspectives: The 50th Anniversary of Missouri’s Sunshine Law',
 'A record-setting number of women will serve in state legislatures in 2023 • Missouri Independent',
 'New York OKs human composting law; 6th state in US to do so',
 'Legacy of Mathews celebrated by the many lives he touched',
 'Cole County judge blocks effort to deny Medicaid funds to Planned Parenthood • Missouri Independent',
 'Legacy of Mathews celebrated by many lives he touched',
 'White House resumes handing out free COVID-19 rapid test kits',
 'White House resumes handing out free COVID-19 rapid test kits • Missouri Independent',
 'How to stay proactive about your health with multi-cancer early detection screening',
 "The 50th Anniversary of Missouri's Sunshine Law",
 'A record-setting number of women will serve in state legislatures in 2023',
 'New York OKs human composting, becomes 6th state in US to do so',
 'Stay proactive about your health with multi-cancer early detection screening',
 'Cole County judge blocks effort to deny Medicaid funds to Planned Parenthood']

In [102]:
republished_content_fuzzy = data_cleaned[data_cleaned['title'].isin(fuzzy_matches)]

In [111]:
republished_content_fuzzy

#440 articles were shared by at least two publications but the titles were slightly different

,Unnamed: 0,author,crawl_datetime,date,hostname,id,image,inferred_tags_set1,meta_data,news,tags,title,url
6,6,Rudi Keller,2023-01-02 13:31:58.462438,2022-12-19 00:00:00,www.kbia.org,3a700da000a2ff2a49c40c332bad9691ef11ddaac6235a...,https://npr.brightspotcdn.com/dims4/default/94...,"['sander', 'majority', 'recognize', 'samesex',...",{'article': {'modified_time': '2022-12-19T19:4...,With the right to same-sex marriage now protec...,NaN,Missouri Republican pushes for constitutional ...,https://www.kbia.org/missouri-news/2022-12-19/...
256,256,Nick Ingram,2023-01-02 13:19:41.876991,2022-12-28 19:18:00,www.kq2.com,9642c9c7a1bfb08063b86f69f467511b291b471a5bd1ff...,https://bloximages.newyork1.vip.townnews.com/k...,"['notification', 'devices', 'saving', 'problem...","{'apple-mobile-web-app-capable': 'yes', 'apple...","(TOPEKA, Ks.) Kansas Governor Laura Kelly sign...","['kansas', ' legislation', ' laura kelly', ' e...",Kansas governor bans TikTok from state-owned d...,https://www.kq2.com/news/kansas-governor-bans-...
375,375,Janice Ellis,2023-01-02 13:20:14.938654,2022-12-26 13:10:28+00:00,www.thepitchkc.com,28c100d997d037572ffdc70bc3c1f2b23435eb22005b01...,https://www.thepitchkc.com/content/uploads/202...,"['things', 'commit', 'political', 'daily', 'gr...","{'HandheldFriendly': 'True', 'MobileOptimized'...",In the midst of holiday festivities with famil...,NaN,"During the holiday season, make time to assess...",https://www.thepitchkc.com/during-the-holiday-...
376,376,Rebecca Rivas,2023-01-02 13:20:15.067850,2022-12-17 20:24:41+00:00,www.thepitchkc.com,3ce3a08e3b12c850bafa617891791ac8778c92c812b1e7...,https://www.thepitchkc.com/content/uploads/202...,"['evidence', 'mason', 'st', 'homicide', 'detec...","{'HandheldFriendly': 'True', 'MobileOptimized'...",The homicide detective who led the investigati...,NaN,Homicide detective in Lamar Johnson’s case say...,https://www.thepitchkc.com/homicide-detective-...
379,379,Rebecca Rivas,2023-01-02 13:20:15.441497,2022-12-15 14:36:43+00:00,www.thepitchkc.com,2a9d870d50a8ee39c11174d0323538dc0ae71bb97e79ed...,https://www.thepitchkc.com/content/uploads/202...,"['innocence', 'attorney', 'judge', 'mason', 's...","{'HandheldFriendly': 'True', 'MobileOptimized'...","After being imprisoned for nearly 30 years, La...",NaN,Lamar Johnson takes the stand to argue his inn...,https://www.thepitchkc.com/lamar-johnson-takes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13111,13111,Rayos Syndication User,2023-01-02 15:22:18.517307,None,www.koamnewsnow.com,47689730d1620a6bf9f551b4a027452878c449c7ee487b...,https://wpcdn.us-east-1.vip.tn-cloud.net/www.k...,"['trumps', 'statement', 'suspend', 'lawler', '...","{'HandheldFriendly': 'True', 'MobileOptimized'...",FILE - Former President Donald Trump announces...,NaN,Trump rebuked for call to suspend Constitution...,https://www.koamnewsnow.com/i/trump-rebuked-fo...
22474,22474,The Associated Press,2023-01-05 18:28:19.701927,2022-12-05 00:00:00,www.fultonsun.com,4dbef2a9b2bee239e9c993d6f7e162f03e3d5fad3351f8...,https://wehco.media.clients.ellingtoncms.com/i...,"['house', 'president', 'suspend', 'trumps', '2...","{'apple-mobile-web-app-title': 'Fulton Sun', '...",WASHINGTON (AP) -- Former President Donald Tru...,NaN,Trump rebuked for call to suspend Constitution...,https://www.fultonsun.com/news/2022/dec/05/tru...
27170,27170,[],2023-01-06 20:10:02.963861,2022-12-04 00:00:00,www.ky3.com,d6dd16f4de5f5ad051658fd308132731989ce579455e97...,https://gray-ky3-prod.cdn.arcpublishing.com/re...,"['statement', 'trumps', 'lawler', 'trump', 'sa...","{'apple-itunes-app': 'app-id=1102844867', 'app...",WASHINGTON (AP) — Former President Donald Trum...,NaN,Trump rebuked for call to suspend Constitution...,https://www.ky3.com/2022/12/04/trump-rebuked-c...
28117,28117,Rebecca Rivas,2023-01-06 20:32:23.668271,2022-12-07 19:35:00,www.lincolnnewsnow.com,615be400fe48d015af55abc1d43da463a925de083f4902...,https://bloximages.newyork1.vip.townnews.com/l...,"['united', 'procedural', 'stat

In [10]:
#Loading the full data and getting the headlines in a list

data = pd.read_csv('december_full_data.csv')